In [1]:
import os, time, math
import math
from math import sqrt
from ann_visualizer.visualize import ann_viz
import numpy as np
import pandas as pd
import seaborn as sns
from keras.layers import Dense ,Dropout,BatchNormalization
from keras.models import Model
#from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
%matplotlib inline
from matplotlib import pyplot as plt
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model, tree, ensemble
import sklearn.metrics as metrics 
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
#from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from xgboost import XGBRegressor
import math
from tune_sklearn import TuneGridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model, tree, ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
reg_decision_model=DecisionTreeRegressor()
from sklearn.model_selection import StratifiedKFold
from hyperopt import tpe,hp,Trials
import xgboost as xgb
from hyperopt.fmin import fmin

In [2]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
# Helper Function
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f}.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def abline(slope, intercept):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')

In [4]:
data=pd.read_csv("E:/Bike Fusion/New Plan/Circular Automatic Collected data for modeling/Buffer Paper/R_exported_buffer_paper_2019_Six_cities_data.csv")
data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
0,1,P001BEN,Colorado Ave.,permanent,2019,Bend,2.0,46.209524,6.561644,2.616438,...,3056.0,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341,1
1,2,P002BEN,Franklin Ave. Both Sides,permanent,2019,Bend,2.0,220.426625,6.958904,3.068493,...,3056.0,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931,2
2,3,P003BEN,Galveston Ave. Both Sides,permanent,2019,Bend,3.0,127.953968,13.726027,4.369863,...,3056.0,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863,3
3,4,P004BEN,Newport Ave. Both Sides,permanent,2019,Bend,1.0,109.378321,7.027397,2.178082,...,3056.0,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368,4
4,5,P005BEN,Portland Ave. Both Sides,permanent,2019,Bend,2.0,61.295238,5.260274,1.698630,...,3056.0,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038,5


In [5]:
track=data[['site_id','site_name','region', 'AADBT']]
track['index']=track.index
track.head()

C:\Users\mxm5116\AppData\Local\Temp/ipykernel_14036/2363949684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track['index']=track.index


,site_id,site_name,region,AADBT,index
0,P001BEN,Colorado Ave.,Bend,46.209524,0
1,P002BEN,Franklin Ave. Both Sides,Bend,220.426625,1
2,P003BEN,Galveston Ave. Both Sides,Bend,127.953968,2
3,P004BEN,Newport Ave. Both Sides,Bend,109.378321,3
4,P005BEN,Portland Ave. Both Sides,Bend,61.295238,4


In [6]:
# Clean the data
an_data=data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['travel_pattern', 'Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Ori

In [7]:
rf_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
rf_data=rf_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
rf_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,8.067776,10.090865,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,7.870166,9.944457,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,8.048469,10.738085,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,7.600402,9.683263,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,7.235619,9.069843,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038


In [8]:
#corrMatrix = rf_data.corr()
#pd.DataFrame.to_csv(corrMatrix,'F:/Bike Fusion/New Plan/Circular Automatic Collected data for modeling/Buffer Paper/corrMatrix_AN_all.csv')

In [9]:
# rf_data1=rf_data[['AADBT','stv_c_adb','stl_raw','log_stv_stl','primary_binary','secondary_binary','tertiary_binary','residential_binary','path_binary',
# 'cycleway_binary','footway_binary','cycleway_lane_binary','Distance.to.Commercial.Area','Distance.to.Industrial.Area','Distance.to.Residential.Center',
# 'Distance.to.Retail.Center','Distance.to.Grass.Center','Distance.to.Park.Center','Distance.to.Water.Body','Distance.to.Forest.Center'
# ,'min_dist_to_school','Point.Speed','Bike.Commuter_tm','Intersection_Density_tm',
# 'Industrial.Area_qm_net','Residential_Road_otm_net','Grass.Area_hm_net','Park.Area_ohm','Water.Area_qm','Bicycle.Parking_tm_net','Bus.Stops_tm',
# 'School_tm','college_tm_net','University_tm','OSM_node_Density_tm','Primary_ohm','Secondary_tm','Tertiary_tm','Path_hm_net','Footway_tm',
# 'Student.Access_tm','pct_at_least_college_education_tm_net','Median_HH_income_otm','population_density_tm','employment_density_tm',
# 'Temp_avg_otm','Hum_avg_otm','Precipitation_otm','Number.of.jobs_tm_net','cycleway_lane_all_om','cycleway_track_all_tm','slope_ohm',
# 'pct_white_om','maxspeed_ohm_net','pct_male_ohm','sep_bikeway_otm_net','BikeFac_binary','BikeFac_ohm_net','BikeFac_onstreet_om',
# 'Distance_to_CBD_mi']]
#'Distance.to.CBD','Distance_to_Water_Body_mi'

# All buffer Sizes

In [37]:
d1=rf_data.loc[:,~rf_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,0,1,0,2.023089,1.285490,1.598419,8.067776,10.090865,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,0,0,0,2.074291,1.403273,1.587276,7.870166,9.944457,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,0,0,0,2.689617,1.680802,2.337582,8.048469,10.738085,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,0,0,0,2.082860,1.156278,1.766325,7.600402,9.683263,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,0,0,1,1.834224,0.992744,1.517683,7.235619,9.069843,0.559726,0.005038


In [38]:
d8=rf_data.loc[:,rf_data.columns.str.contains('_net', case=False)]
d8.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
0,35.531170,0.678076,1.908397,24.365364,503.063397,0.0,0.0,339.145348,0.0,10949.72823,...,35001.51393,60971.18678,116621.72730,0.395530,1.440465,3.565952,9.355694,15.616797,33.203417,57.739067
1,39.005450,1.704244,4.369246,55.724012,326.972002,0.0,0.0,2609.755647,0.0,0.00000,...,21772.75413,52411.79064,112209.70950,0.336709,1.189527,2.748207,7.121148,12.788902,27.702900,58.699022
2,45.759783,2.886810,6.308618,128.518935,489.712962,0.0,0.0,2489.827234,0.0,0.00000,...,25354.47209,67283.59369,109300.85250,0.069645,0.311978,2.213339,5.935694,12.814490,29.903869,54.308968
3,42.307763,0.927522,2.192320,35.713162,346.534723,0.0,0.0,1274.619412,0.0,0.00000,...,20762.70934,47873.90545,105821.84770,0.081332,0.577948,3.120323,7.120087,13.484632,29.851699,57.632743
4,21.726393,1.662712,7.652959,82.319830,346.565625,0.0,0.0,196.668745,0.0,0.00000,...,14571.25813,46345.18729,89953.08027,0.278187,1.237188,3.608611,7.176205,12.228374,31.115168,53.971717


In [39]:
rf_data1=pd.concat([d7, d8.reindex(d7.index)], axis=1)
#rf_data1=pd.concat([d7,d8], axis=1)
rf_data1.shape

(312, 381)

In [16]:
corrMatrix1 = rf_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college',
       ...
       'Path_tm_net', 'Footway_tm_net', 'cycleway_track_all_tm_net',
       'Student.Access_tm_net', 'pct_at_least_college_education_tm_net',
       'HH_density_tm_net', 'population_density_tm_net',
       'employment_density_tm_net', 'Number.of.jobs_tm_net',
       'sep_bikeway_otm_net'],
      dtype='object', length=142)

In [32]:
data_all=rf_data1[top_corr_features]
data_all1=data_all.drop(['pct_at_least_college_education_ohm_net','Percentage.of.Bike.Commuter_tm_net','pct_at_least_college_education_tm_net','stv_adb','stv_nc_adb','stl_raw','OSM_node_Density_tm_net','OSM_node_Density_qm_net','OSM_node_Density_hm_net','OSM_node_Density_tfm_net' ],axis=1)
data_all1.shape                     

(312, 132)

In [33]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data_all1.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data_all1[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 15 s


,feature,VIF
2,tertiary_binary,1.299356
15,Footway_otm_net,1.882840
21,Industrial.Area_tfm_net,2.040668
32,Footway_tm_net,2.101860
3,path_binary,2.147517
19,Primary_hm_net,2.194969
26,cycleway_track_all_tfm_net,2.211348
4,Distance.to.forest,2.216610
33,sep_bikeway_otm_net,2.232138
13,Bus.Stops_otm_net,2.346483


In [34]:
vif_data['feature']

1                                 stv_c_adb
2                           tertiary_binary
3                               path_binary
4                        Distance.to.forest
5                       min_dist_to_college
6                            log_stv_nc_adb
7                               log_stl_raw
8                        Distance_to_CBD_mi
9                     Bike.Commuter_otm_net
10      Percentage.of.Bike.Commuter_otm_net
11                  Industrial.Area_otm_net
12                 Residential_Road_otm_net
13                        Bus.Stops_otm_net
14                             Path_otm_net
15                          Footway_otm_net
16                   Bicycle.Parking_qm_net
17                              Path_qm_net
18    pct_at_least_college_education_qm_net
19                           Primary_hm_net
20                    Number.of.jobs_hm_net
21                  Industrial.Area_tfm_net
22                           School_tfm_net
23                          coll

# Now lets select 0.1 mile buffer variabe

In [40]:
# rf_data2=rf_data1.loc[:,~rf_data1.columns.str.contains('qm', case=False)] 
# rf_data3=rf_data2.loc[:,~rf_data2.columns.str.contains('hm', case=False)] 
# rf_data4=rf_data3.loc[:,~rf_data3.columns.str.contains('tfm', case=False)] 
# rf_data5=rf_data4.loc[:,~rf_data4.columns.str.contains('om', case=False)] 
# rf_data6=rf_data5.loc[:,~rf_data5.columns.str.contains('ohm', case=False)] 
# rf_data7=rf_data6.loc[:,~rf_data6.columns.str.contains('tm', case=False)]
# keep the data for one-tenth mile buffer only
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('otm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data9.columns.tolist()

['AADBT',
 'stv_adb',
 'stv_c_adb',
 'stl_raw',
 'primary_binary',
 'secondary_binary',
 'tertiary_binary',
 'residential_binary',
 'path_binary',
 'cycleway_binary',
 'footway_binary',
 'cycleway_lane_binary',
 'cycleway_track_all_binary',
 'lanes',
 'Distance.to.Commercial.Area',
 'Distance.to.Commercial.Area.Center',
 'Distance.to.Industrial.Area',
 'Distance.to.Industrial.Center',
 'Distance.to.Residential.Area',
 'Distance.to.Residential.Center',
 'Distance.to.Retail.Area',
 'Distance.to.Retail.Center',
 'Distance.to.Grass',
 'Distance.to.Grass.Center',
 'Distance.to.Park',
 'Distance.to.Park.Center',
 'Distance.to.Water.Body',
 'Distance.to.Water.Center',
 'Distance.to.forest',
 'Distance.to.Forest.Center',
 'Distance.to.CBD',
 'min_dist_to_school',
 'min_dist_to_college',
 'min_dist_to_university',
 'Point.Speed',
 'Point.Bridge',
 'stv_nc_adb',
 'sep_bikeway_binary',
 'sep_onstreet_binary',
 'BikeFac_binary',
 'arterial_binary',
 'arterial_no_bike_lane_binary',
 'primary_no_bik

In [41]:
corrMatrix1 = rf_data9.corr()
corrMatrix1

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
AADBT,1.000000,0.691349,0.734261,0.563353,-0.089504,-0.089987,-0.215239,0.060670,0.259477,0.089271,...,0.102878,0.121742,0.123075,0.101100,-0.011952,0.198497,0.128525,0.166397,0.210284,0.124372
stv_adb,0.691349,1.000000,0.763331,0.460712,-0.045819,-0.002297,-0.095177,0.024144,0.171482,-0.032325,...,0.048287,0.034556,0.027782,0.003473,-0.008305,0.280032,-0.077144,0.058352,-0.003829,0.020197
stv_c_adb,0.734261,0.763331,1.000000,0.614351,-0.072056,0.041851,-0.167485,0.175909,0.272769,-0.095544,...,0.172326,0.187032,0.187665,0.096134,-0.076818,0.090904,0.088822,0.009130,0.008388,0.095561
stl_raw,0.563353,0.460712,0.614351,1.000000,0.035996,0.163431,-0.186925,-0.128496,0.312011,-0.077424,...,0.192919,0.160908,0.165479,0.159288,-0.107054,0.021405,0.067893,-0.018278,0.098532,0.303417
primary_binary,-0.089504,-0.045819,-0.072056,0.035996,1.000000,-0.084963,-0.085827,-0.097803,-0.032947,-0.084096,...,-0.081267,-0.083325,-0.084265,0.080511,-0.005158,-0.074718,-0.128732,-0.124083,-0.102400,-0.025570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Temp_avg_otm_net,0.198497,0.280032,0.090904,0.021405,-0.074718,-0.163984,-0.218600,-0.083534,0.162517,0.220024,...,0.098457,0.076753,0.030549,0.071518,-0.045124,1.000000,0.026786,0.743313,0.139801,-0.065802
Hum_avg_otm_net,0.128525,-0.077144,0.088822,0.067893,-0.128732,-0.095121,-0.026264,0.122074,-0.044897,0.191368,...,0.248185,0.302856,0.354898,0.006352,-0.115439,0.026786,1.000000,0.617939,0.149012,0.165828
Precipitation_otm_net,0.166397,0.058352,0.009130,-0.018278,-0.124083,-0.149883,-0.159386,-0.071206,0.047262,0.323510,...,0.149374,0.162972,0.158294,0.099227,-0.111155,0.743313,0.617939,1.000000,0.266968,0.103844
sep_bikeway_otm_net,0.210284,-0.003829,0.008388,0.098532,-0.102400,-0.027382,-0.187770,-0.282012,0.258994,0.404535,...,-0.064963,-0.066028,-0.079024,0.275054,-0.027869,0.139801,0.149012,0.266968,1.000000,0.590291


In [42]:
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [72]:
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Percentage.of.Bike.Commuter_otm', 'Industrial.Area_otm',
       'Bus.Stops_otm', 'OSM_node_Density_otm', 'Footway_otm', 'bridge_otm',
       'pct_at_least_college_education_otm'],
      dtype='object')

In [48]:
data1=rf_data9[top_corr_features]
data11=data1.drop(['Distance.to.CBD','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','log_stv_nc_adb','stl_raw'],axis=1)

In [49]:
# rf_data99=rf_data[['AADBT','stv_c_adb', 'stl_raw', 'tertiary_binary',
#        'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
#         'min_dist_to_college', 'min_dist_to_university',
#        'stv_nc_adb', 
#        'tertiary_bike_lane_binary','log_stv_stl', 'Distance_to_CBD_mi',
#        'Percentage.of.Bike.Commuter_otm', 'Industrial.Area_otm',
#        'Bus.Stops_otm', 'OSM_node_Density_otm', 'Footway_otm', 'bridge_otm',
#        'pct_at_least_college_education_otm', 'Bike.Commuter_otm_net',
#        'Percentage.of.Bike.Commuter_otm_net', 'Industrial.Area_otm_net',
#        'Residential_Road_otm_net', 'Bus.Stops_otm_net', 'Path_otm_net',
#        'Footway_otm_net', 'sep_bikeway_otm_net']]

In [50]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data11[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 80.4 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
15,Footway_otm_net,1.208846
2,tertiary_binary,1.232121
6,stv_nc_adb,1.335372
16,sep_bikeway_otm_net,1.716438
13,Bus.Stops_otm_net,1.722264
3,path_binary,1.777276
4,Distance.to.forest,1.809913
14,Path_otm_net,2.045679
10,Percentage.of.Bike.Commuter_otm_net,2.092548
9,Bike.Commuter_otm_net,2.298927


In [51]:
vif_data['feature']

1                               stv_c_adb
2                         tertiary_binary
3                             path_binary
4                      Distance.to.forest
5                     min_dist_to_college
6                              stv_nc_adb
7                             log_stl_raw
8                      Distance_to_CBD_mi
9                   Bike.Commuter_otm_net
10    Percentage.of.Bike.Commuter_otm_net
11                Industrial.Area_otm_net
12               Residential_Road_otm_net
13                      Bus.Stops_otm_net
14                           Path_otm_net
15                        Footway_otm_net
16                    sep_bikeway_otm_net
Name: feature, dtype: object

# Now select 0.25 mile buffer

In [53]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('qm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data9.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.836151,2036.203607,1862.396701,829.005006,1.856461,48.478082,61.352877,0.028959,3168.931840,1.440465
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1084.679453,2295.157044,1901.477295,1351.149336,1.720310,48.478082,61.352877,0.028959,924.377060,1.189527
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1503.180649,2744.512703,2440.557228,387.590203,1.412683,48.478082,61.352877,0.028959,1647.243197,0.311978
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1249.236493,2808.364961,2215.072908,1496.004378,1.812751,48.478082,61.352877,0.028959,1111.749342,0.577948
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,902.528117,2145.488551,1717.049191,941.551494,0.000000,48.478082,61.352877,0.028959,1428.936640,1.237188


In [54]:
corrMatrix1 = rf_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Bike.Commuter_qm_net', 'Percentage.of.Bike.Commuter_qm_net',
       'bike.commuter.density_qm_net', 'Industrial.Area_qm_net',
       'Residential_Road_qm_net', 'Bicycle.Parking_qm_net', 'Bus.Stops_qm_net',
       'OSM_node_Density_qm_net', 'Path_qm_net', 'Footway_qm_net',
       'pct_at_least_college_education_qm_net', 'Number.of.jobs_qm_net'],
      dtype='object')

In [55]:
data2=rf_data9[top_corr_features]
data22=data2.drop(['tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','log_stv_nc_adb','stl_raw','log_stv_adb'],axis=1)

In [56]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data22.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data22[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 110 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
2,tertiary_binary,1.214474
16,Path_qm_net,1.253751
17,Footway_qm_net,1.345836
18,pct_at_least_college_education_qm_net,1.421716
6,stv_nc_adb,1.430171
3,path_binary,1.473355
15,OSM_node_Density_qm_net,1.481140
19,Number.of.jobs_qm_net,1.580049
4,Distance.to.forest,1.895614
13,Bicycle.Parking_qm_net,2.145544


In [57]:
vif_data['feature']

1                                 stv_c_adb
2                           tertiary_binary
3                               path_binary
4                        Distance.to.forest
5                       min_dist_to_college
6                                stv_nc_adb
7                               log_stl_raw
8                        Distance_to_CBD_mi
9                      Bike.Commuter_qm_net
10       Percentage.of.Bike.Commuter_qm_net
11                   Industrial.Area_qm_net
12                  Residential_Road_qm_net
13                   Bicycle.Parking_qm_net
14                         Bus.Stops_qm_net
15                  OSM_node_Density_qm_net
16                              Path_qm_net
17                           Footway_qm_net
18    pct_at_least_college_education_qm_net
19                    Number.of.jobs_qm_net
Name: feature, dtype: object

# Run 0.5 mile buffer

In [81]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('hm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data10=rf_data9.loc[:,~rf_data9.columns.str.contains('ohm_net', case=False)]
rf_data10.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.075347,2029.870332,1849.840318,3179.167538,1.796814,48.478082,61.352877,0.028959,8226.788063,3.565952
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1140.645925,2363.272923,1975.098575,6298.912469,1.717111,48.478082,61.352877,0.028959,2418.948981,2.748207
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1316.888252,2592.371997,2251.099836,1822.446713,1.598114,48.478082,61.352877,0.028959,8156.981385,2.213339
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1273.780756,2877.745161,2276.266710,5575.592943,1.996023,48.478082,61.352877,0.028959,6657.914592,3.120323
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1004.362315,2397.361344,1920.692370,4137.955371,0.000000,48.478082,61.352877,0.028959,4029.161672,3.608611


In [82]:
corrMatrix1 = rf_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_hm_net', 'Bike.Commuter_hm_net',
       'Percentage.of.Bike.Commuter_hm_net', 'bike.commuter.density_hm_net',
       'Industrial.Area_hm_net', 'Residential_Road_hm_net',
       'Bicycle.Parking_hm_net', 'Bus.Stops_hm_net', 'OSM_node_Density_hm_net',
       'Primary_hm_net', 'Path_hm_net', 'Footway_hm_net',
       'pct_at_least_college_education_hm_net', 'HH_density_hm_net',
       'population_density_hm_net', 'employment_density_hm_net',
       'Number.of.jobs_hm_net'],
      dtype='object')

In [83]:
data3=rf_data10[top_corr_features]
data33=data3.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','tertiary_no_bike_lane_binary','stl_raw','tertiary_bike_lane_binary','OSM_node_Density_hm_net'],axis=1)

In [84]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data33.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data33[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 120 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
2,tertiary_binary,1.264972
3,path_binary,1.464916
15,Path_hm_net,1.515377
17,pct_at_least_college_education_hm_net,1.517598
14,Primary_hm_net,1.742690
6,stv_nc_adb,1.750457
4,Distance.to.forest,1.930762
16,Footway_hm_net,1.969602
19,Number.of.jobs_hm_net,1.988960
10,Industrial.Area_hm_net,2.315084


In [85]:
vif_data['feature']

1                                 stv_c_adb
2                           tertiary_binary
3                               path_binary
4                        Distance.to.forest
5                       min_dist_to_college
6                                stv_nc_adb
7                               log_stv_stl
8                        Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_hm_net
10                   Industrial.Area_hm_net
11                  Residential_Road_hm_net
12                   Bicycle.Parking_hm_net
13                         Bus.Stops_hm_net
14                           Primary_hm_net
15                              Path_hm_net
16                           Footway_hm_net
17    pct_at_least_college_education_hm_net
18                        HH_density_hm_net
19                    Number.of.jobs_hm_net
Name: feature, dtype: object

# 0.75 mile buffer variable selections

In [66]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('tfm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data10=rf_data9.loc[:,~rf_data9.columns.str.contains('ohm_net', case=False)]
rf_data10.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1023.544330,1999.166320,1801.170916,8961.622695,1.718788,48.478082,61.352877,0.028959,24512.96424,9.355694
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1185.968034,2434.314963,2041.267896,9175.566610,1.809894,48.478082,61.352877,0.028959,11881.27513,7.121148
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1217.786570,2640.446631,2202.291352,5881.091374,1.820654,48.478082,61.352877,0.028959,12950.78401,5.935694
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1262.446464,2830.839076,2276.899147,7704.876164,2.141138,48.478082,61.352877,0.028959,11469.32877,7.120087
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1098.051723,2586.992843,2077.994523,8422.905274,0.000000,48.478082,61.352877,0.028959,7652.72686,7.176205


In [67]:
corrMatrix1 = rf_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.2]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data9[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_tfm_net', 'Bike.Commuter_tfm_net',
       'Percentage.of.Bike.Commuter_tfm_net', 'bike.commuter.density_tfm_net',
       'Intersection_Density_tfm_net', 'Industrial.Area_tfm_net',
       'Bicycle.Parking_tfm_net', 'Bus.Stops_tfm_net', 'School_tfm_net',
       'college_tfm_net', 'OSM_node_Density_tfm_net', 'Primary_tfm_net',
       'Tertiary_tfm_net', 'Path_tfm_net', 'Footway_tfm_net',
       'cycleway_track_all_tfm_net', 'Student.Access_tfm_net',
       'pct_at_least_college_education_tfm_net', 'HH_density_tfm_net',
       'population_density_tfm_net', 'e

In [68]:
data4=rf_data10[top_corr_features]
data44=data4.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb','stl_raw'],axis=1)

In [69]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data44.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data44[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 271 ms


,feature,VIF
2,tertiary_binary,1.242158
11,Industrial.Area_tfm_net,1.487768
6,stv_nc_adb,1.535487
3,path_binary,1.580674
17,Path_tfm_net,1.630783
21,pct_at_least_college_education_tfm_net,1.662673
19,cycleway_track_all_tfm_net,1.918288
4,Distance.to.forest,2.022537
15,Primary_tfm_net,2.101064
10,Intersection_Density_tfm_net,2.329651


In [70]:
vif_data['feature']

1                                  stv_c_adb
2                            tertiary_binary
3                                path_binary
4                         Distance.to.forest
5                        min_dist_to_college
6                                 stv_nc_adb
7                                log_stl_raw
8                         Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_tfm_net
10              Intersection_Density_tfm_net
11                   Industrial.Area_tfm_net
12                   Bicycle.Parking_tfm_net
13                            School_tfm_net
14                           college_tfm_net
15                           Primary_tfm_net
16                          Tertiary_tfm_net
17                              Path_tfm_net
18                           Footway_tfm_net
19                cycleway_track_all_tfm_net
20                    Student.Access_tfm_net
21    pct_at_least_college_education_tfm_net
22                    Number.of.jobs_tfm_net
Name: feat

# Run 1.0 mile buffer Model

In [71]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('om_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data9.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,987.765909,2039.114372,1773.560854,14210.18096,1.807770,48.478082,61.352877,0.028959,35001.51393,15.616797
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1174.910427,2479.059314,2071.134552,12359.63894,1.742249,48.478082,61.352877,0.028959,21772.75413,12.788902
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1107.086579,2486.103335,2043.481168,11142.91564,2.240183,48.478082,61.352877,0.028959,25354.47209,12.814490
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1193.187839,2686.377224,2187.958431,11303.31561,2.256581,48.478082,61.352877,0.028959,20762.70934,13.484632
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1152.249511,2637.286420,2140.344673,11169.65567,0.000000,48.478082,61.352877,0.028959,14571.25813,12.228374


In [74]:
corrMatrix1 = rf_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data12[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_om_net', 'Bike.Commuter_om_net',
       'Percentage.of.Bike.Commuter_om_net', 'bike.commuter.density_om_net',
       'Intersection_Density_om_net', 'Bicycle.Parking_om_net',
       'Bus.Stops_om_net', 'School_om_net', 'college_om_net',
       'OSM_node_Density_om_net', 'Primary_om_net', 'Tertiary_om_net',
       'Path_om_net', 'Footway_om_net', 'cycleway_track_all_om_net',
       'Student.Access_om_net', 'pct_at_least_college_education_om_net',
       'HH_density_om_net', 'population_density_om_net',
       'employment_density_om_net', 'Number.of.jobs_om_

In [78]:
data5=rf_data9[top_corr_features]
data55=data5.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'stl_raw','log_stl_raw','OSM_node_Density_om_net'],axis=1)

In [79]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data55.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data55[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 218 ms


,feature,VIF
6,tertiary_bike_lane_binary,1.250499
2,path_binary,1.584608
16,Path_om_net,1.666823
5,stv_nc_adb,1.825127
20,pct_at_least_college_education_om_net,2.000294
3,Distance.to.forest,2.100243
18,cycleway_track_all_om_net,2.271877
15,Tertiary_om_net,2.783814
4,min_dist_to_college,2.872086
17,Footway_om_net,2.906141


In [80]:
vif_data['feature']

1                                 stv_c_adb
2                               path_binary
3                        Distance.to.forest
4                       min_dist_to_college
5                                stv_nc_adb
6                 tertiary_bike_lane_binary
7                               log_stv_stl
8                        Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_om_net
10              Intersection_Density_om_net
11                         Bus.Stops_om_net
12                            School_om_net
13                           college_om_net
14                           Primary_om_net
15                          Tertiary_om_net
16                              Path_om_net
17                           Footway_om_net
18                cycleway_track_all_om_net
19                    Student.Access_om_net
20    pct_at_least_college_education_om_net
Name: feature, dtype: object

# Run 1.5 mile Buffer 

In [86]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('ohm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data9.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.789068,2430.079342,1990.766003,20294.00738,2.193156,48.478082,61.352877,0.028959,60971.18678,33.203417
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1179.750560,2658.042382,2181.243968,21062.80977,2.041506,48.478082,61.352877,0.028959,52411.79064,27.702900
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,939.174351,2153.735239,1762.012857,19474.65755,2.427138,48.478082,61.352877,0.028959,67283.59369,29.903869
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1085.433314,2467.190081,2013.320626,19438.67456,2.233351,48.478082,61.352877,0.028959,47873.90545,29.851699
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1212.423151,2759.914670,2242.972630,17313.71673,0.000000,48.478082,61.352877,0.028959,46345.18729,31.115168


In [87]:
corrMatrix1 = rf_data9.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data6[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_ohm_net', 'Bike.Commuter_ohm_net',
       'Percentage.of.Bike.Commuter_ohm_net', 'bike.commuter.density_ohm_net',
       'Intersection_Density_ohm_net', 'Bicycle.Parking_ohm_net',
       'Bus.Stops_ohm_net', 'School_ohm_net', 'college_ohm_net',
       'OSM_node_Density_ohm_net', 'Primary_ohm_net', 'Tertiary_ohm_net',
       'Path_ohm_net', 'Footway_ohm_net', 'cycleway_track_all_ohm_net',
       'Student.Access_ohm_net', 'pct_at_least_college_education_ohm_net',
       'HH_density_ohm_net', 'population_density_ohm_net',
       'employment_density_ohm_net',

In [91]:
data6=rf_data9[top_corr_features]
data66=data6.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','stl_raw','OSM_node_Density_ohm_net'],axis=1)

In [92]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data66.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data66[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 245 ms


,feature,VIF
2,tertiary_binary,1.192703
3,path_binary,1.522137
6,stv_nc_adb,1.812216
14,Path_ohm_net,1.885227
16,cycleway_track_all_ohm_net,1.965197
18,pct_at_least_college_education_ohm_net,2.099614
4,Distance.to.forest,2.137975
15,Footway_ohm_net,2.406632
5,min_dist_to_college,2.778206
1,stv_c_adb,3.131789


In [93]:
vif_data['feature']

1                                  stv_c_adb
2                            tertiary_binary
3                                path_binary
4                         Distance.to.forest
5                        min_dist_to_college
6                                 stv_nc_adb
7                                log_stv_stl
8                         Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_ohm_net
10              Intersection_Density_ohm_net
11                            School_ohm_net
12                           Primary_ohm_net
13                          Tertiary_ohm_net
14                              Path_ohm_net
15                           Footway_ohm_net
16                cycleway_track_all_ohm_net
17                    Student.Access_ohm_net
18    pct_at_least_college_education_ohm_net
Name: feature, dtype: object

# Run 2.0 mile buffer

In [1254]:
# Run 2 mile Buffer RF Model

In [96]:
rf_data8=rf_data.loc[:,rf_data.columns.str.contains('tm_net', case=False)]
rf_data9=pd.concat([d7, rf_data8.reindex(d7.index)], axis=1)
rf_data10=rf_data9.loc[:,~rf_data9.columns.str.contains('_otm_net', case=False)]
rf_data10.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1026.478817,2415.698097,1941.406570,26072.11710,2.264271,48.478082,61.352877,0.028959,116621.72730,57.739067
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1179.759608,2740.534260,2212.566542,26705.89684,2.200950,48.478082,61.352877,0.028959,112209.70950,58.699022
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,942.623989,2212.217243,1789.363360,25163.68821,2.390589,48.478082,61.352877,0.028959,109300.85250,54.308968
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1082.118678,2504.794925,2029.680644,26006.37448,2.361176,48.478082,61.352877,0.028959,105821.84770,57.632743
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1170.006244,2688.548616,2183.688248,25701.46188,0.000000,48.478082,61.352877,0.028959,89953.08027,53.971717


In [97]:
corrMatrix1 = rf_data10.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(rf_data6[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.forest', 'Distance.to.Forest.Center',
       'Distance.to.CBD', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_tm_net', 'Bike.Commuter_tm_net',
       'Percentage.of.Bike.Commuter_tm_net', 'bike.commuter.density_tm_net',
       'Intersection_Density_tm_net', 'Bicycle.Parking_tm_net',
       'Bus.Stops_tm_net', 'School_tm_net', 'college_tm_net',
       'OSM_node_Density_tm_net', 'Primary_tm_net', 'Tertiary_tm_net',
       'Path_tm_net', 'Footway_tm_net', 'cycleway_track_all_tm_net',
       'Student.Access_tm_net', 'pct_at_least_college_education_tm_net',
       'HH_density_tm_net', 'population_density_tm_net',
       'employment_density_tm_net', 'Number.of.jobs_tm_

In [104]:
data7=rf_data10[top_corr_features]
data77=data7.drop(['stv_adb','log_stv_adb', 'log_stv_c_adb','log_stv_nc_adb', 'log_stl_raw','stl_raw','OSM_node_Density_tm_net'],axis=1)

In [105]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data77.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data77[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 270 ms


,feature,VIF
2,tertiary_binary,1.192016
3,path_binary,1.459478
6,stv_nc_adb,1.834682
12,Path_tm_net,1.879951
14,cycleway_track_all_tm_net,1.978937
4,Distance.to.forest,2.155277
10,Intersection_Density_tm_net,2.176060
16,pct_at_least_college_education_tm_net,2.223178
13,Footway_tm_net,2.466327
5,min_dist_to_college,2.793420


In [106]:
vif_data['feature'][1:]

2                           tertiary_binary
3                               path_binary
4                        Distance.to.forest
5                       min_dist_to_college
6                                stv_nc_adb
7                               log_stv_stl
8                        Distance_to_CBD_mi
9        Percentage.of.Bike.Commuter_tm_net
10              Intersection_Density_tm_net
11                           Primary_tm_net
12                              Path_tm_net
13                           Footway_tm_net
14                cycleway_track_all_tm_net
15                    Student.Access_tm_net
16    pct_at_least_college_education_tm_net
Name: feature, dtype: object

# Portland Generalized Modal data

In [10]:
P_data1=data[data['region']=='Portland']
P_data1.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
224,225,P001POR,HAWTHORNE BR north side,permanent,2019,Portland,1.0,1775.363690,109.219178,80.945205,...,303222.0,2.532664,6.985360,24.117618,6.985360,66.650573,113.195792,0.315109,0.049033,225
225,226,P002POR,HAWTHORNE BR south side,permanent,2019,Portland,2.0,1537.211111,111.027397,74.904110,...,304246.0,0.000000,5.366635,23.795831,5.366635,70.113770,113.414613,0.403667,0.111882,226
226,227,P003POR,"SW MOODY AVE, WEST SIDE CYCLE TRACK",permanent,2019,Portland,1.0,1346.261310,98.109589,59.917808,...,259355.0,0.000000,0.027482,13.117664,0.027482,49.214257,81.480702,1.005728,0.848626,227
227,228,P004POR,STEEL BR lower deck,permanent,2019,Portland,1.0,1951.052146,128.041096,84.410959,...,310223.0,0.187564,3.716239,11.537631,3.716239,57.695648,111.057680,1.028972,0.015873,228
228,229,P005POR,BROADWAY BR north side,permanent,2019,Portland,1.0,1182.406584,120.712329,82.164384,...,300850.0,0.000000,0.000000,7.536517,0.000000,37.898346,107.614898,1.277080,0.154749,229


In [11]:
# Clean the data
an_data=P_data1._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.route_otm', 'Bike.Share.route_qm', 'Bike.Share.route_hm', 'Bike.Share.route_tfm', 'Bike.Share.route_om', 'Bike.Share.route_ohm', 'Bike.Share.route_tm', 'Bike.Share.route_otm_net', 'Bike.Share.route_qm_net', 'Bike.Share.route_hm_net', 'Bike.Share.route_tfm_net', 'Bike.Share.route_om_net', 'Bike.Share.route_ohm_net', 'Bike.Share.route_tm_net']


In [12]:
p1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
p1_data=p1_data.drop(['Unnamed: 0','travel_pattern','year','latitude','longitude','rowIndex','Bike.Share.Origin_otm_net' ,'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net','Bike.Share.Destination_qm_net','Bike.Share.Origin_qm_net','Bike.Share.Crossing_qm_net','Bike.Share.Origin_hm_net',
       'Bike.Share.Destination_hm_net', 'Bike.Share.Crossing_hm_net'],axis=1)
p1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Bike.Share.best_ohm,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,283530.0,303222.0,2.532664,6.985360,24.117618,6.985360,66.650573,113.195792,0.315109,0.049033
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,280829.0,304246.0,0.000000,5.366635,23.795831,5.366635,70.113770,113.414613,0.403667,0.111882
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,175720.0,259355.0,0.000000,0.027482,13.117664,0.027482,49.214257,81.480702,1.005728,0.848626
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,287107.0,310223.0,0.187564,3.716239,11.537631,3.716239,57.695648,111.057680,1.028972,0.015873
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,272401.0,300850.0,0.000000,0.000000,7.536517,0.000000,37.898346,107.614898,1.277080,0.154749


In [13]:
d1=p1_data.loc[:,~p1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,0,0,0,4.702471,4.406051,3.376699,9.201905,13.904376,0.315109,0.049033
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,0,0,0,4.718743,4.329471,3.614244,8.804475,13.523219,0.403667,0.111882
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,0,0,0,4.596226,4.109526,3.668467,8.606485,13.202711,1.005728,0.848626
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,0,0,0,4.860131,4.447474,3.798409,9.759848,14.619979,1.028972,0.015873
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,0,0,0,4.801660,4.420819,3.677514,8.996900,13.798560,1.277080,0.154749


In [14]:
#P_data=P_data.loc[:,~P_data.columns.str.contains('_net', case=False)]
#P_data.shape

In [15]:
net_data=p1_data.loc[:,p1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
224,16.022713,0.273407,1.706372,16.176254,532.489169,36280.00139,89978.92067,4896.216136,0.0,20394.26036,...,47870.89106,71203.37451,92941.98212,1.029416,2.060420,6.218730,14.880263,25.866599,44.845269,66.712079
225,47.072918,1.599795,3.398547,97.474784,731.154518,29457.33066,209066.79190,5395.197227,0.0,20394.26036,...,48511.37404,71761.60733,92295.70529,0.770046,2.164560,6.949038,15.206040,26.320156,44.574785,66.429052
226,38.106851,0.919089,2.411874,39.503692,214.906743,0.00000,0.00000,0.000000,0.0,0.00000,...,36123.49685,63170.31750,78966.18338,0.529213,1.310920,3.628847,8.939504,15.394518,29.958385,46.407092
227,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.0,0.00000,...,31829.98229,53582.69558,79526.20183,0.000000,1.265361,5.128670,12.501881,23.230175,48.835481,68.410953
228,14.240723,3.681275,25.850340,215.845855,410.434122,59216.95686,0.00000,0.000000,0.0,39191.61447,...,17955.08952,38452.80702,72864.35922,1.285570,2.643968,7.859357,13.841898,22.789381,45.119279,64.196776


In [16]:
P_data=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,47870.89106,71203.37451,92941.98212,1.029416,2.060420,6.218730,14.880263,25.866599,44.845269,66.712079
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,48511.37404,71761.60733,92295.70529,0.770046,2.164560,6.949038,15.206040,26.320156,44.574785,66.429052
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,36123.49685,63170.31750,78966.18338,0.529213,1.310920,3.628847,8.939504,15.394518,29.958385,46.407092
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,31829.98229,53582.69558,79526.20183,0.000000,1.265361,5.128670,12.501881,23.230175,48.835481,68.410953
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,17955.08952,38452.80702,72864.35922,1.285570,2.643968,7.859357,13.841898,22.789381,45.119279,64.196776


# All Buffer

In [244]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
corrMatrix1.head(20)

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
AADBT,1.000000,0.794337,0.820675,0.577469,NaN,-0.039008,-0.254606,-0.059604,0.449860,0.148836,...,0.489468,0.560954,0.538833,0.320648,0.320753,0.364367,0.380629,0.395912,0.467138,0.442822
stv_adb,0.794337,1.000000,0.981994,0.604640,NaN,0.197546,-0.278858,-0.145738,0.354220,0.067629,...,0.349441,0.356481,0.402324,0.434060,0.385436,0.338027,0.324552,0.313717,0.381326,0.371701
stv_c_adb,0.820675,0.981994,1.000000,0.596049,NaN,0.154720,-0.279389,-0.106741,0.368529,0.039612,...,0.354421,0.379488,0.461359,0.418677,0.375290,0.355019,0.358366,0.360175,0.441140,0.448281
stl_raw,0.577469,0.604640,0.596049,1.000000,NaN,0.238255,-0.195422,-0.499147,0.557755,0.095455,...,0.506128,0.425612,0.296963,0.484358,0.505787,0.551420,0.542288,0.563386,0.547971,0.374147
primary_binary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
secondary_binary,-0.039008,0.197546,0.154720,0.238255,NaN,1.000000,-0.178174,-0.516398,-0.127515,-0.102869,...,-0.187452,-0.158159,-0.154544,0.135128,0.117218,-0.068227,-0.139953,-0.153669,-0.108168,-0.115884
tertiary_binary,-0.254606,-0.278858,-0.279389,-0.195422,NaN,-0.178174,1.000000,-0.414039,-0.102240,-0.082479,...,0.145553,0.228475,0.178223,-0.016071,-0.015322,-0.017609,0.029093,0.040763,0.022644,0.048432
residential_binary,-0.059604,-0.145738,-0.106741,-0.499147,NaN,-0.516398,-0.414039,1.000000,-0.296319,-0.239046,...,-0.439072,-0.374566,-0.160365,-0.535016,-0.546378,-0.449964,-0.402854,-0.392271,-0.315818,-0.156003
path_binary,0.449860,0.354220,0.368529,0.557755,NaN,-0.127515,-0.102240,-0.296319,1.000000,-0.059028,...,0.414232,0.293604,0.130727,0.574363,0.522438,0.535039,0.482084,0.465000,0.380801,0.201893
cycleway_binary,0.148836,0.067629,0.039612,0.095455,NaN,-0.102869,-0.082479,-0.239046,-0.059028,1.000000,...,0.454280,0.312134,0.132240,0.274560,0.296212,0.269947,0.272032,0.243883,0.166395,0.086527


In [246]:
pdata0=P_data[top_corr_features]
data00=pdata0.drop(['log_stv_nc_adb','stv_nc_adb','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','Distance.to.Grass.Center'],axis=1)

In [247]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = data00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = data00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 23 s


,feature,VIF
3,min_dist_to_college,2.055326
15,slope_tm_net,2.122319
6,Industrial.Area_ohm_net,2.160579
2,Distance.to.Grass,2.351790
14,Median_HH_income_tm_net,2.521958
1,stv_c_adb,2.663159
16,sep_bikeway_qm_net,2.732459
5,Percentage.of.Bike.Commuter_ohm_net,2.738938
18,BikeFac_otm_net,2.768421
4,log_stl_raw,3.396016


In [248]:
vif_data['feature']

1                               stv_c_adb
2                       Distance.to.Grass
3                     min_dist_to_college
4                             log_stl_raw
5     Percentage.of.Bike.Commuter_ohm_net
6                 Industrial.Area_ohm_net
7                     Retail.Area_ohm_net
8                        Tertiary_ohm_net
9                         Footway_ohm_net
10                      pct_white_ohm_net
11                 Student.Access_ohm_net
12                      Grass.Area_tm_net
13                      Water.Area_tm_net
14                Median_HH_income_tm_net
15                           slope_tm_net
16                     sep_bikeway_qm_net
17                    sep_bikeway_ohm_net
18                        BikeFac_otm_net
Name: feature, dtype: object

# Portland 0.1 mile buffer

In [214]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,1420.611648,3747.528119,3600.904001,85.296136,0.000000,54.164932,69.331507,0.070274,3895.491784,1.029416
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,3183.046616,6315.327332,5967.143063,705.855457,0.000000,54.164932,69.331507,0.070274,2594.714346,0.770046
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,1413.016667,2160.548064,1999.494552,20.924795,1.422465,54.164932,69.331507,0.070274,2794.245012,0.529213
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,54.164932,69.331507,0.070274,0.000000,0.000000
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,836.876035,1473.053293,1389.744366,261.750545,0.000000,54.164932,69.331507,0.070274,1218.749097,1.285570


In [216]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Industrial.Area_otm_net', 'Residential_Road_otm_net',
       'Water.Area_otm_net', 'OSM_node_Density_otm_net', 'Primary_otm_net',
       'Path_otm_net', 'Cycleway_otm_net', 'bridge_otm_net',
       'pct_female_otm_net', 'pct_at_least_college_education_otm_net',
       'Median.Ag

In [218]:
pdata0=P_data[top_corr_features]
pdata00=pdata0.drop(['stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','Distance.to.Industrial.Center','Distance.to.Grass.Center','Distance.to.Grass','log_stv_nc_adb','stv_nc_adb'],axis=1)

In [219]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = pdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 264 ms


,feature,VIF
14,Water.Area_otm_net,1.403708
1,tertiary_binary,1.586776
22,HH_density_otm_net,1.675698
21,Median_HH_income_otm_net,1.937113
3,Distance.to.Commercial.Area.Center,1.962382
5,Distance.to.Residential.Area,1.990301
17,Cycleway_otm_net,2.033213
6,Distance.to.Retail.Area,2.049174
20,Median.Age_otm_net,2.440982
7,min_dist_to_college,2.490995


In [220]:
vif_data['feature']

1                        tertiary_binary
2                            path_binary
3     Distance.to.Commercial.Area.Center
4            Distance.to.Industrial.Area
5           Distance.to.Residential.Area
6                Distance.to.Retail.Area
7                    min_dist_to_college
8                            Point.Speed
9                           Point.Bridge
10                         log_stv_c_adb
11                           log_stl_raw
12               Industrial.Area_otm_net
13              Residential_Road_otm_net
14                    Water.Area_otm_net
15              OSM_node_Density_otm_net
16                       Primary_otm_net
17                      Cycleway_otm_net
18                        bridge_otm_net
19                    pct_female_otm_net
20                    Median.Age_otm_net
21              Median_HH_income_otm_net
22                    HH_density_otm_net
23                       BikeFac_otm_net
Name: feature, dtype: object

# Portland 0.25 mile buffer

In [259]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,3240.142379,6701.249274,6366.797307,8015.216710,0.000000,54.164932,69.331507,0.070274,6115.874769,2.060420
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,5053.873044,9518.764055,8992.536295,9444.441480,0.000000,54.164932,69.331507,0.070274,6211.854253,2.164560
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,1848.841778,2881.926212,2711.723131,248.342357,2.096974,54.164932,69.331507,0.070274,5933.274552,1.310920
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,3896.558018,5949.595338,5762.898128,1.784506,0.000000,54.164932,69.331507,0.070274,4422.774761,1.265361
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,836.876035,1473.053293,1389.744366,2258.354159,0.000000,54.164932,69.331507,0.070274,1796.562880,2.643968


In [260]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Industrial.Area_qm_net', 'Residential_Road_qm_net',
       'OSM_node_Density_qm_net', 'Primary_qm_net', 'Secondary_qm_net',
       'Tertiary_qm_net', 'Path_qm_net', 'Cycleway_qm_net', 'bridge_qm_net',
       'Median_HH_income_qm_net', 'population_density_qm_net',
       'employm

In [261]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['tertiary_no_bike_lane_binary','stv_nc_adb','Distance.to.Industrial.Center','Distance.to.Industrial.Center','Distance.to.Grass','Distance.to.Grass.Center','Distance.to.CBD','tertiary_bike_lane_binary','log_stv_nc_adb'],axis=1)
#pdata11=pdata1.drop(['travel_pattern','Bike.Share.best_otm','Bike.Share.Crossing_otm','Bike.Share.Origin_otm_net','Bike.Share.Destination_otm_net','Bike.Share.Crossing_otm_net','Distance.to.CBD','log_stv_adb' ,'log_stv_c_adb','log_stv_nc_adb','log_stl_raw','stl_raw','tertiary_no_bike_lane_binary','Path_otm_net','stv_adb','tertiary_bike_lane_binary','population_density_otm_net','population_density_otm','Distance.to.Grass.Center','sep_bikeway_otm','sep_bikeway_otm_net','sep_bikeway_binary','Primary_otm_net','Industrial.Area_otm_net','BikeFac_otm_net','Residential_Road_otm','log_stv_stl','bridge_otm','BikeFac_binary','employment_density_otm_net','Point.Bridge','stv_nc_adb'],axis=1)
pdata11.shape

(88, 37)

In [262]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 242 ms


,feature,VIF
13,Industrial.Area_qm_net,1.765189
5,Distance.to.Residential.Area,1.795240
20,employment_density_qm_net,1.821620
19,Median_HH_income_qm_net,1.858518
3,Distance.to.Commercial.Area.Center,1.859532
6,Distance.to.Retail.Area,1.877614
16,Secondary_qm_net,2.143260
1,tertiary_binary,2.237145
14,OSM_node_Density_qm_net,2.456708
7,min_dist_to_college,2.503698


In [263]:
vif_data['feature']

1                        tertiary_binary
2                            path_binary
3     Distance.to.Commercial.Area.Center
4            Distance.to.Industrial.Area
5           Distance.to.Residential.Area
6                Distance.to.Retail.Area
7                    min_dist_to_college
8                            Point.Speed
9                           Point.Bridge
10                        BikeFac_binary
11                         log_stv_c_adb
12                           log_stl_raw
13                Industrial.Area_qm_net
14               OSM_node_Density_qm_net
15                        Primary_qm_net
16                      Secondary_qm_net
17                           Path_qm_net
18                         bridge_qm_net
19               Median_HH_income_qm_net
20             employment_density_qm_net
21                 Number.of.jobs_qm_net
Name: feature, dtype: object

# Portland 0.5 mile Buffer

In [297]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,6448.573643,10639.372370,10210.350130,41670.950430,0.000000,54.164932,69.331507,0.070274,17034.864960,6.218730
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,6763.009156,11195.976830,10724.031520,41958.970900,0.000000,54.164932,69.331507,0.070274,17512.878270,6.949038
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,2429.641594,3821.993202,3616.063077,1787.748645,2.243655,54.164932,69.331507,0.070274,9458.086618,3.628847
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,4625.172978,7133.994981,6911.832983,3413.134005,0.000000,54.164932,69.331507,0.070274,11644.433000,5.128670
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,2137.025398,3506.022360,3326.291797,5903.012503,0.000000,54.164932,69.331507,0.070274,7338.333731,7.859357


In [302]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Commercial.Area_hm_net',
       'Industrial.Area_hm_net', 'Grass.Area_hm_net',
       'OSM_node_Density_hm_net', 'Primary_hm_net', 'Secondary_hm_net',
       'Tertiary_hm_net', 'Path_hm_net', 'bridge_hm_net', 'pct_male_hm_net',
       'pct_female_hm_net', 'pct_at_least_college_education_hm_net',
       'Number.of.jobs_hm_net', 'sep_bikeway_hm_net', 'BikeFac_hm_net'],
      dtype='object'

In [303]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','stv_nc_adb','Distance.to.Grass.Center','log_stv_stl',
       ],axis=1)

In [304]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stv_c_adb,4.632390
2,tertiary_binary,2.186788
3,path_binary,3.290702
4,Distance.to.Commercial.Area.Center,2.798491
5,Distance.to.Industrial.Area,3.388638
6,Distance.to.Industrial.Center,4.512941
7,Distance.to.Residential.Center,2.454323
8,Distance.to.Grass,3.317177
9,Point.Bridge,3.627152
10,BikeFac_binary,2.836289


In [305]:
vif_data['feature']

1                                 stv_c_adb
2                           tertiary_binary
3                               path_binary
4        Distance.to.Commercial.Area.Center
5               Distance.to.Industrial.Area
6             Distance.to.Industrial.Center
7            Distance.to.Residential.Center
8                         Distance.to.Grass
9                              Point.Bridge
10                           BikeFac_binary
11                       Distance_to_CBD_mi
12                   Commercial.Area_hm_net
13                   Industrial.Area_hm_net
14                        Grass.Area_hm_net
15                  OSM_node_Density_hm_net
16                           Primary_hm_net
17                         Secondary_hm_net
18                            bridge_hm_net
19                        pct_female_hm_net
20    pct_at_least_college_education_hm_net
Name: feature, dtype: object

# 0.75 mile buffer

In [17]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,Bike.Share.Origin_tfm_net,Bike.Share.Destination_tfm_net,Bike.Share.Crossing_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,71515.035470,0.000000,54.164932,69.331507,0.070274,80143.0,82171.0,143345.0,30182.03249,14.880263
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,69922.418150,0.000000,54.164932,69.331507,0.070274,78634.0,80329.0,138744.0,30051.30576,15.206040
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,8415.878491,2.606877,54.164932,69.331507,0.070274,20408.0,21668.0,37057.0,22632.07626,8.939504
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,19983.220760,0.000000,54.164932,69.331507,0.070274,41095.0,46440.0,97247.0,18341.91114,12.501881
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,18389.499280,0.000000,54.164932,69.331507,0.070274,32686.0,35719.0,93831.0,12792.53157,13.841898


In [18]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Industrial.Area_tfm_net',
       'Retail.Area_tfm_net', 'Grass.Area_tfm_net', 'Bicycle.Parking_tfm_net',
       'Bus.Stops_tfm_net', 'OSM_node_Density_tfm_net', 'Primary_tfm_net',
       'Secondary_tfm_net', 'Tertiary_tfm_net', 'Path_tfm_net',
       'Cycleway_tfm_net', 'pct_male_tfm_net', 'pct_female_tfm_net',
       'pct_at_least_college_education_tfm_net', 'Number.of.jobs_tfm_net',
  

In [19]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['log_stv_nc_adb','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','Distance.to.Grass','Point.Bridge','sep_bikeway_binary','stv_nc_adb','Distance.to.Grass.Center'],axis=1)

In [20]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,tertiary_binary,1.982484
2,path_binary,2.270701
3,Distance.to.Commercial.Area.Center,3.204013
4,Distance.to.Industrial.Area,4.411840
5,Distance.to.Industrial.Center,4.100302
6,Distance.to.Residential.Center,2.117785
7,BikeFac_binary,3.507680
8,log_stv_c_adb,3.857890
9,log_stl_raw,4.384661
10,Distance_to_CBD_mi,4.200875


In [21]:
vif_data['feature']

1                            tertiary_binary
2                                path_binary
3         Distance.to.Commercial.Area.Center
4                Distance.to.Industrial.Area
5              Distance.to.Industrial.Center
6             Distance.to.Residential.Center
7                             BikeFac_binary
8                              log_stv_c_adb
9                                log_stl_raw
10                        Distance_to_CBD_mi
11                   Industrial.Area_tfm_net
12                       Retail.Area_tfm_net
13                        Grass.Area_tfm_net
14                   Bicycle.Parking_tfm_net
15                           Primary_tfm_net
16                         Secondary_tfm_net
17                          Cycleway_tfm_net
18                        pct_female_tfm_net
19    pct_at_least_college_education_tfm_net
Name: feature, dtype: object

# portland 1 mile buffer

In [22]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,Bike.Share.Origin_om_net,Bike.Share.Destination_om_net,Bike.Share.Crossing_om_net,sep_bikeway_om_net,BikeFac_om_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,95531.20810,0.000000,54.164932,69.331507,0.070274,110631.0,114551.0,181389.0,47870.89106,25.866599
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,94782.79291,0.000000,54.164932,69.331507,0.070274,114646.0,119118.0,181297.0,48511.37404,26.320156
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,24663.91522,3.000628,54.164932,69.331507,0.070274,37712.0,35642.0,62715.0,36123.49685,15.394518
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,56903.59100,0.000000,54.164932,69.331507,0.070274,79868.0,89525.0,162188.0,31829.98229,23.230175
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,57395.13872,0.000000,54.164932,69.331507,0.070274,75113.0,82416.0,149975.0,17955.08952,22.789381


In [23]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary', 'BikeFac_binary',
       'tertiary_no_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Industrial.Area_om_net',
       'Retail.Area_om_net', 'Grass.Area_om_net', 'Bicycle.Parking_om_net',
       'college_om_net', 'OSM_node_Density_om_net', 'Primary_om_net',
       'Secondary_om_net', 'Tertiary_om_net', 'Path_om_net', 'Cycleway_om_net',
       'cycleway_lane_all_om_net', 'pct_male_om_net', 'pct_female_om_net',
       'Student.Access_om_net', 'pct_at_least_college_education_

In [29]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['Distance.to.Grass.Center','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','stv_nc_adb','log_stv_stl'],axis=1)

In [30]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stv_c_adb,3.895576
2,tertiary_binary,1.870438
3,path_binary,2.534728
4,Distance.to.Commercial.Area.Center,2.529990
5,Distance.to.Industrial.Center,2.832208
6,Distance.to.Residential.Center,2.236702
7,Distance.to.Grass,2.551241
8,Point.Bridge,3.408497
9,BikeFac_binary,2.698462
10,log_stl_raw,4.233875


In [31]:
vif_data['feature']

1                                 stv_c_adb
2                           tertiary_binary
3                               path_binary
4        Distance.to.Commercial.Area.Center
5             Distance.to.Industrial.Center
6            Distance.to.Residential.Center
7                         Distance.to.Grass
8                              Point.Bridge
9                            BikeFac_binary
10                              log_stl_raw
11                   Industrial.Area_om_net
12                       Retail.Area_om_net
13                        Grass.Area_om_net
14                         Secondary_om_net
15                          Cycleway_om_net
16                        pct_female_om_net
17                    Student.Access_om_net
18    pct_at_least_college_education_om_net
Name: feature, dtype: object

# Portland 1.5 Mile Buffer

In [32]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,Bike.Share.Origin_ohm_net,Bike.Share.Destination_ohm_net,Bike.Share.Crossing_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,128354.13880,0.000000,54.164932,69.331507,0.070274,198346.0,204588.0,257779.0,71203.37451,44.845269
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,126522.78830,0.000000,54.164932,69.331507,0.070274,189810.0,198825.0,252371.0,71761.60733,44.574785
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,74949.21042,2.717951,54.164932,69.331507,0.070274,82299.0,80741.0,129257.0,63170.31750,29.958385
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,146818.78500,0.000000,54.164932,69.331507,0.070274,182777.0,195684.0,256785.0,53582.69558,48.835481
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,132184.73760,0.000000,54.164932,69.331507,0.070274,181700.0,190911.0,245765.0,38452.80702,45.119279


In [33]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Bike.Commuter_ohm_net', 'Percentage.of.Bike.Commuter_ohm_net',
       'bike.commuter.density_ohm_net', 'Intersection_Density_ohm_net',
       'Industrial.Area_ohm_net', 'Retail.Area_ohm_net', 'Grass.Area_ohm_net',
       'Bicycle.Parking_ohm_net', 'Bus.Stops_ohm_net', 'college_o

In [35]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['Distance.to.Industrial.Center','Distance.to.Commercial.Area','Distance.to.Commercial.Area.Center','Distance.to.Grass','Distance.to.Grass.Center','stl_raw','tertiary_no_bike_lane_binary','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','tertiary_bike_lane_binary','stv_nc_adb','log_stv_stl'],axis=1)

In [36]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stv_c_adb,3.607775
2,tertiary_binary,1.715484
3,Distance.to.Industrial.Area,4.277366
4,Distance.to.Residential.Area,1.699032
5,Distance.to.Retail.Area,1.566176
6,min_dist_to_college,2.881222
7,sep_bikeway_binary,3.535067
8,BikeFac_binary,4.442489
9,Distance_to_CBD_mi,3.675936
10,Percentage.of.Bike.Commuter_ohm_net,2.866818


In [37]:
vif_data['feature']

1                               stv_c_adb
2                         tertiary_binary
3             Distance.to.Industrial.Area
4            Distance.to.Residential.Area
5                 Distance.to.Retail.Area
6                     min_dist_to_college
7                      sep_bikeway_binary
8                          BikeFac_binary
9                      Distance_to_CBD_mi
10    Percentage.of.Bike.Commuter_ohm_net
11                Industrial.Area_ohm_net
12                    Retail.Area_ohm_net
13                     Grass.Area_ohm_net
14                       Tertiary_ohm_net
15                        Footway_ohm_net
16                      pct_white_ohm_net
17                     pct_female_ohm_net
18                 Student.Access_ohm_net
19                          slope_ohm_net
20                    sep_bikeway_ohm_net
Name: feature, dtype: object

# Portland 2.0 mile buffer

In [38]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
P_data=pd.concat([d7, d1.reindex(d7.index)], axis=1)
P_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,Bike.Share.Origin_tm_net,Bike.Share.Destination_tm_net,Bike.Share.Crossing_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
224,1775.363690,109.219178,80.945205,9915,0,0,0,0,1,0,...,174473.5583,0.000000,54.164932,69.331507,0.070274,251849.0,255848.0,291946.0,92941.98212,66.712079
225,1537.211111,111.027397,74.904110,6663,0,0,0,0,1,0,...,179720.9011,0.000000,54.164932,69.331507,0.070274,252113.0,256070.0,291840.0,92295.70529,66.429052
226,1346.261310,98.109589,59.917808,5466,0,0,0,0,0,1,...,146909.2310,2.711647,54.164932,69.331507,0.070274,157148.0,166870.0,217805.0,78966.18338,46.407092
227,1951.052146,128.041096,84.410959,17323,0,0,0,0,1,0,...,185285.2828,0.000000,54.164932,69.331507,0.070274,262380.0,263102.0,295889.0,79526.20183,68.410953
228,1182.406584,120.712329,82.164384,8077,0,0,0,0,1,0,...,181314.3749,0.000000,54.164932,69.331507,0.070274,249868.0,248271.0,284971.0,72864.35922,64.196776


In [39]:
corrMatrix1 = P_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Retail.Area',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.CBD',
       'min_dist_to_college', 'Point.Speed', 'Point.Bridge', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Total.Commuter_tm_net', 'Bike.Commuter_tm_net',
       'Percentage.of.Bike.Commuter_tm_net', 'bike.commuter.density_tm_net',
       'Intersection_Density_tm_net', 'Grass.Area_tm_net', 'Water.Area_tm_net',
       'Bicycle.Parking_tm_net', 'Bus.Stops_tm_net', 'School_tm_net',
    

In [42]:
pdata1=P_data[top_corr_features]
pdata11=pdata1.drop(['tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','Distance.to.Grass','stl_raw','log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw','stv_adb','Distance.to.CBD','stv_nc_adb','log_stv_stl','Distance.to.Grass.Center'],axis=1)

In [43]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

pdata11
X_dataf = pdata11.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = pdata11[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stv_c_adb,4.699630
2,tertiary_binary,1.612291
3,path_binary,4.075619
4,Distance.to.Commercial.Area.Center,2.114624
5,Distance.to.Industrial.Area,4.644215
6,Distance.to.Industrial.Center,4.665850
7,Distance.to.Residential.Area,1.819946
8,Distance.to.Retail.Area,1.569906
9,min_dist_to_college,3.484374
10,sep_bikeway_binary,4.471226


In [44]:
vif_data['feature']

1                              stv_c_adb
2                        tertiary_binary
3                            path_binary
4     Distance.to.Commercial.Area.Center
5            Distance.to.Industrial.Area
6          Distance.to.Industrial.Center
7           Distance.to.Residential.Area
8                Distance.to.Retail.Area
9                    min_dist_to_college
10                    sep_bikeway_binary
11                        BikeFac_binary
12    Percentage.of.Bike.Commuter_tm_net
13                     Grass.Area_tm_net
14                     Water.Area_tm_net
15                         School_tm_net
16                       Cycleway_tm_net
17                      pct_white_tm_net
18               Median_HH_income_tm_net
19                          slope_tm_net
Name: feature, dtype: object

# Eugene all bufferes

In [47]:
e_data=data[data['region']=='Eugene']
e_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
147,148,P001EUG,Fern Ridge Path west of Chambers,permanent,2019,Eugene,2.0,513.408379,12.575342,3.780822,...,147890.0,2.57938,10.240767,38.828042,81.100254,109.983487,177.408835,1.582742,1.272067,148
148,149,P002EUG,Millrace Path @ Booth Kelly,permanent,2019,Eugene,3.0,109.807738,7.780822,2.397260,...,2374.0,0.00000,0.000000,7.688736,32.359430,60.783955,87.937981,3.765385,0.896833,149
149,150,P003EUG,Southbank Path west of Knickerbocker Bridge,permanent,2019,Eugene,2.0,330.297326,10.383562,4.863014,...,182248.0,0.00000,2.580734,52.336410,100.643748,160.837403,250.348289,1.934576,0.329983,150
150,151,P004EUG,North Bank Path south of Greenway Bridge,permanent,2019,Eugene,3.0,429.732467,19.589041,7.342466,...,83568.0,0.06550,26.300129,75.951888,133.116175,253.846733,388.687995,1.526089,0.171903,151
151,152,P005EUG,Alder north of 18th Ave,permanent,2019,Eugene,1.0,410.282143,6.972603,4.479452,...,192315.0,0.00000,0.000000,2.284680,20.413902,48.177451,241.359538,0.917704,0.394418,152


In [48]:
# Clean the data
an_data=e_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

[]


In [65]:
e1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
e1_data=e1_data.drop(['Unnamed: 0','year','travel_pattern','latitude','longitude','rowIndex','Bike.Share.Origin_otm',
 'Bike.Share.Destination_otm',
 'Bike.Share.Crossing_otm','Bike.Share.Origin_qm',
 'Bike.Share.Destination_qm',
 'Bike.Share.Crossing_qm','Bike.Share.Origin_tfm',
 'Bike.Share.Destination_tfm',
 'Bike.Share.Crossing_tfm','Bike.Share.Origin_om',
 'Bike.Share.Destination_om',
 'Bike.Share.Crossing_om','Bike.Share.Origin_ohm',
 'Bike.Share.Destination_ohm',
 'Bike.Share.Crossing_ohm','Bike.Share.Origin_tm',
 'Bike.Share.Destination_tm',
 'Bike.Share.Crossing_tm','Bike.Share.Origin_qm_net',
 'Bike.Share.Destination_qm_net',
 'Bike.Share.Crossing_qm_net','Bike.Share.Origin_otm_net',
 'Bike.Share.Destination_otm_net',
 'Bike.Share.Crossing_otm_net','Bike.Share.Origin_hm_net',
 'Bike.Share.Destination_hm_net',
 'Bike.Share.Crossing_hm_net','Bike.Share.Origin_tfm_net',
 'Bike.Share.Destination_tfm_net',
 'Bike.Share.Crossing_tfm_net','Bike.Share.Origin_om_net',
 'Bike.Share.Destination_om_net',
 'Bike.Share.Crossing_om_net','Bike.Share.Origin_ohm_net',
 'Bike.Share.Destination_ohm_net',
 'Bike.Share.Crossing_ohm_net', 'Bike.Share.Origin_tm_net',
 'Bike.Share.Destination_tm_net',
 'Bike.Share.Crossing_tm_net','Bike.Share.best_otm',
 'Bike.Share.best_qm',
 'Bike.Share.best_hm',
 'Bike.Share.best_tfm',
 'Bike.Share.best_om',
 'Bike.Share.best_ohm',
 'Bike.Share.best_tm','Bike.Share.Origin_hm',
 'Bike.Share.Destination_hm',
 'Bike.Share.Crossing_hm','Bike.Share.route_otm_net',
'Bike.Share.route_otm','Bike.Share.route_qm','Bike.Share.route_tfm','Bike.Share.route_qm_net',
'Bike.Share.route_hm','Bike.Share.route_om','Bike.Share.route_otm_net','Bike.Share.route_qm_net','Bike.Share.route_hm_net','Bike.Share.route_tfm_net','Bike.Share.route_om_net','Bike.Share.route_ohm_net','Bike.Share.route_tm_net'],axis=1)

e1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,6.893656,9.501911,2.57938,10.240767,38.828042,81.100254,109.983487,177.408835,1.582742,1.272067
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,5.318120,7.490690,0.00000,0.000000,7.688736,32.359430,60.783955,87.937981,3.765385,0.896833
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,7.925519,10.357689,0.00000,2.580734,52.336410,100.643748,160.837403,250.348289,1.934576,0.329983
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,7.118016,10.142775,0.06550,26.300129,75.951888,133.116175,253.846733,388.687995,1.526089,0.171903
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,7.675546,9.751557,0.00000,0.000000,2.284680,20.413902,48.177451,241.359538,0.917704,0.394418


In [66]:
d1=e1_data.loc[:,~e1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,0,0,0,2.608255,1.564612,2.281823,6.893656,9.501911,1.582742,1.272067
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,0,0,0,2.172570,1.222969,1.853726,5.318120,7.490690,3.765385,0.896833
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,0,0,0,2.432170,1.768664,1.874958,7.925519,10.357689,1.934576,0.329983
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,0,0,0,3.024759,2.121359,2.583739,7.118016,10.142775,1.526089,0.171903
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,0,0,0,2.076011,1.701005,1.250804,7.675546,9.751557,0.917704,0.394418


In [67]:
net_data=e1_data.loc[:,e1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
147,78.307904,9.264077,11.830322,638.918474,137.934622,121158.3469,0.0,168.349020,0.0000,0.000000,...,28544.49543,41592.31486,86955.45017,0.502687,2.182450,6.963112,11.338760,16.209625,31.157091,64.614200
148,9.700128,0.454217,4.682586,41.519315,182.817150,0.0000,0.0,1143.351480,0.0000,0.000000,...,21846.48920,43380.68769,133682.92850,0.108875,0.186416,2.941423,5.014374,10.026571,27.171879,61.216622
149,22.019848,3.028648,13.754173,250.174645,0.000000,0.0000,0.0,0.000000,0.0000,72662.979610,...,72252.69708,137974.29400,191007.29350,0.333792,1.138297,3.272191,7.531423,15.894440,36.479439,57.279205
150,8.053182,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,356953.4949,0.000000,...,32880.55259,60210.99915,113423.05130,0.263303,1.252938,3.727162,10.160410,16.171030,32.882896,64.485386
151,207.040059,16.607024,8.021165,530.000563,191.485443,0.0000,0.0,1182.936629,0.0000,3588.039399,...,102933.75180,230364.07390,423183.61790,0.925463,3.156309,11.013575,26.109649,45.140987,92.258786,154.650904


In [68]:
e_data1=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,28544.49543,41592.31486,86955.45017,0.502687,2.182450,6.963112,11.338760,16.209625,31.157091,64.614200
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,21846.48920,43380.68769,133682.92850,0.108875,0.186416,2.941423,5.014374,10.026571,27.171879,61.216622
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,72252.69708,137974.29400,191007.29350,0.333792,1.138297,3.272191,7.531423,15.894440,36.479439,57.279205
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,32880.55259,60210.99915,113423.05130,0.263303,1.252938,3.727162,10.160410,16.171030,32.882896,64.485386
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,102933.75180,230364.07390,423183.61790,0.925463,3.156309,11.013575,26.109649,45.140987,92.258786,154.650904


In [69]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.40]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'Distance.to.Park.Center',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'log_stv_adb', 'log_stv_c_adb', 'log_stl_raw',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Grass.Area_otm_net',
       'Bicycle.Parking_otm_net', 'Bus.Stops_otm_net', 'University_otm_net',
       'Footway_otm_net', 'pct_at_least_college_education_otm_net',
       'Median_HH_income_otm_net', 'Grass.Area_qm_net', 'University_qm_net',
       'Cycleway_qm_net', 'Footway_qm_net', 'Median_HH_income_qm_net',
       'Grass.Area_hm_net', 'Water.Area_hm_net', 'University_hm_net',
       'Path_hm_net', 'Cycleway_hm_net', 'Footway_hm_net',
       'Student.Access_hm_net', 'Median.Age_hm_net', 'Median_HH_income_hm_net',
       'Grass.Area_tfm_net', 'Water.Area_tfm_net', 'University_tfm_net',
       'Path_tfm_net', 'Cycleway_tfm_net', 'Footway_tfm_net',
       'pct_African_American_tfm_net', 'Student.Access_tfm_ne

In [70]:
edata0=e_data1[top_corr_features]
#pdata00=pdata0.drop(['log_stl_raw','stv_adb','travel_pattern','Bike.Share.best_otm','Bike.Share.Destination_otm_net','Bike.Share.Crossing_otm_net','Bike.Share.Origin_otm_net','Distance.to.CBD','Path_otm_net','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','employment_density_otm_net','Distance.to.Grass.Center','log_stv_adb','employment_density_otm','sep_bikeway_otm','sep_bikeway_otm_net','stl_raw','Primary_otm','log_stv_c_adb','log_stv_nc_adb','sep_bikeway_binary','Residential_Road_otm','stv_nc_adb','log_stv_stl','BikeFac_otm','Industrial.Area_otm','bridge_otm','Bike.Share.Crossing_otm','BikeFac_binary','OSM_node_Density_otm_net'],axis=1)
edata00=edata0.drop(['stv_c_adb'],axis=1)
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(edata00[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [62]:
%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data

Wall time: 0 ns


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
1,stl_raw,3.749733
2,Distance.to.Park.Center,1.651155
3,Distance.to.Forest.Center,2.687097
4,log_stv_c_adb,2.314872
5,log_stl_raw,4.412383
6,Grass.Area_otm_net,2.531782
7,Bicycle.Parking_otm_net,3.411828
8,Bus.Stops_otm_net,3.758935
9,University_otm_net,2.389264
10,pct_at_least_college_education_otm_net,2.324215


In [63]:
vif_data['feature']

1                                    stl_raw
2                    Distance.to.Park.Center
3                  Distance.to.Forest.Center
4                              log_stv_c_adb
5                                log_stl_raw
6                         Grass.Area_otm_net
7                    Bicycle.Parking_otm_net
8                          Bus.Stops_otm_net
9                         University_otm_net
10    pct_at_least_college_education_otm_net
11                  Median_HH_income_otm_net
12                           Cycleway_qm_net
13                         Grass.Area_hm_net
14                         Water.Area_hm_net
15                         University_hm_net
16              pct_African_American_tfm_net
17                         Water.Area_om_net
18                   Bicycle.Parking_ohm_net
19                         pct_white_ohm_net
20                   Median_HH_income_tm_net
Name: feature, dtype: object

# 0.1 mile Buffer

In [73]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,5573.089529,11866.970660,8770.072904,22.710117,1.737349,52.194247,75.815616,0.099863,1521.281034,0.502687
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,1169.450292,2537.498917,2189.770563,29.699550,0.000000,52.194247,75.815616,0.099863,574.861566,0.108875
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,1053.636132,4676.108072,4505.926451,10.844541,5.405463,52.194247,75.815616,0.099863,1762.423373,0.333792
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,805.855181,1215.201087,1139.607592,200.921896,1.003565,52.194247,75.815616,0.099863,1390.238470,0.263303
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,5307.381943,13267.719360,12660.571300,33.622310,2.612067,52.194247,75.815616,0.099863,994.574294,0.925463


In [74]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Intersection_Density_otm_net',
       'Residential_Road_otm_net', 'Retail.Area_otm_net', 'Grass.Area_otm_net',
       'Forest.Area_otm_net', 'Bicycle.Parking_otm_net', 'Bus.Stops_otm_net',
       'University_otm_net', 'Cycleway_otm_net', 'Footway_otm_net',
       

In [75]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['stv_c_adb','tertiary_no_bike_lane_binary','Distance.to.Grass.Center','Distance.to.Grass','tertiary_bike_lane_binary','Distance.to.Grass.Center'],axis=1)

In [76]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 432 ms


,feature,VIF
1,stl_raw,2.213215
2,tertiary_binary,2.054731
3,Distance.to.Industrial.Area,2.928725
4,Distance.to.Retail.Center,3.518592
5,Distance.to.Park.Center,1.638069
6,Distance.to.Water.Center,3.541167
7,Distance.to.forest,3.007197
8,stv_nc_adb,2.469365
9,Intersection_Density_otm_net,2.579462
10,Residential_Road_otm_net,2.794517


In [77]:
vif_data['feature']

1                                    stl_raw
2                            tertiary_binary
3                Distance.to.Industrial.Area
4                  Distance.to.Retail.Center
5                    Distance.to.Park.Center
6                   Distance.to.Water.Center
7                         Distance.to.forest
8                                 stv_nc_adb
9               Intersection_Density_otm_net
10                  Residential_Road_otm_net
11                       Retail.Area_otm_net
12                        Grass.Area_otm_net
13                       Forest.Area_otm_net
14                   Bicycle.Parking_otm_net
15                         Bus.Stops_otm_net
16                        University_otm_net
17                          Cycleway_otm_net
18                            bridge_otm_net
19                         pct_white_otm_net
20              pct_African_American_otm_net
21                        pct_female_otm_net
22                    Student.Access_otm_net
23    pct_

# Eugene 0.25 mile buffer

In [78]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,4114.514100,8788.423508,6670.243683,502.146780,1.808796,52.194247,75.815616,0.099863,5137.089214,2.182450
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,1219.455759,2644.644397,2278.450432,57.443376,0.000000,52.194247,75.815616,0.099863,574.861566,0.186416
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,834.641393,2822.846049,2683.571604,24.433998,3.304107,52.194247,75.815616,0.099863,6010.207540,1.138297
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,904.816306,1610.620568,1474.327895,791.211629,1.155668,52.194247,75.815616,0.099863,5884.207495,1.252938
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,7274.280868,18023.806980,17238.897170,6921.489501,1.893339,52.194247,75.815616,0.099863,3543.305186,3.156309


In [79]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Residential_Road_qm_net',
       'Grass.Area_qm_net', 'Forest.Area_qm_net', 'Bicycle.Parking_qm_net',
       'Bus.Stops_qm_net', 'University_qm_net', 'Path_qm_net',
       'Cycleway_qm_net', 'Footway_qm_net', 'bridge_qm_net',
       'pct_white_qm_net', 'pct_Africa

In [84]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary'],axis=1)

In [85]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 470 ms


,feature,VIF
5,Distance.to.Park,1.460800
23,slope_qm_net,1.644240
1,stl_raw,2.133500
2,tertiary_binary,2.147322
8,log_stv_nc_adb,2.185597
15,bridge_qm_net,2.265739
20,pct_at_least_college_education_qm_net,2.470894
16,pct_white_qm_net,2.591145
9,Residential_Road_qm_net,2.594668
13,Bus.Stops_qm_net,2.654319


In [86]:
vif_data['feature']

1                                   stl_raw
2                           tertiary_binary
3             Distance.to.Industrial.Center
4                 Distance.to.Retail.Center
5                          Distance.to.Park
6                  Distance.to.Water.Center
7                        Distance.to.forest
8                            log_stv_nc_adb
9                   Residential_Road_qm_net
10                        Grass.Area_qm_net
11                       Forest.Area_qm_net
12                   Bicycle.Parking_qm_net
13                         Bus.Stops_qm_net
14                        University_qm_net
15                            bridge_qm_net
16                         pct_white_qm_net
17              pct_African_American_qm_net
18                        pct_female_qm_net
19                    Student.Access_qm_net
20    pct_at_least_college_education_qm_net
21                        Median.Age_qm_net
22                    Number.of.jobs_qm_net
23                             s

# Eugene 0.5 mile buffer

In [87]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,2906.155416,6253.244819,5027.784990,5639.829828,1.572727,52.194247,75.815616,0.099863,13900.154990,6.963112
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,2072.598340,4480.308313,3798.587520,2422.697157,0.000000,52.194247,75.815616,0.099863,6112.033788,2.941423
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,914.016440,2544.161096,2370.249746,232.872763,2.312213,52.194247,75.815616,0.099863,16796.391010,3.272191
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,984.291353,1884.698634,1690.709610,2733.759127,0.959600,52.194247,75.815616,0.099863,10369.976270,3.727162
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,8240.962302,21025.300550,20197.183880,12069.336150,1.893032,52.194247,75.815616,0.099863,24520.625400,11.013575


In [88]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Percentage.of.Bike.Commuter_hm_net',
       'bike.commuter.density_hm_net', 'Residential_Road_hm_net',
       'Grass.Area_hm_net', 'Water.Area_hm_net', 'Forest.Area_hm_net',
       'Bicycle.Parking_hm_net', 'Bus.Stops_hm_net', 'University_hm_net',
       'Primary_

In [92]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['tertiary_no_bike_lane_binary','stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw'],axis=1)

In [93]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 572 ms


,feature,VIF
3,Distance.to.Park,1.546663
22,slope_hm_net,1.934915
19,pct_white_hm_net,1.962646
6,tertiary_bike_lane_binary,2.061780
7,log_stv_nc_adb,2.112169
8,log_stl_raw,2.255110
12,Water.Area_hm_net,2.261885
13,Forest.Area_hm_net,2.291958
10,Residential_Road_hm_net,2.436606
21,pct_at_least_college_education_hm_net,2.576850


In [94]:
vif_data['feature']

1               Distance.to.Industrial.Area
2                 Distance.to.Retail.Center
3                          Distance.to.Park
4                  Distance.to.Water.Center
5                        Distance.to.forest
6                 tertiary_bike_lane_binary
7                            log_stv_nc_adb
8                               log_stl_raw
9        Percentage.of.Bike.Commuter_hm_net
10                  Residential_Road_hm_net
11                        Grass.Area_hm_net
12                        Water.Area_hm_net
13                       Forest.Area_hm_net
14                   Bicycle.Parking_hm_net
15                        University_hm_net
16                           Primary_hm_net
17                          Cycleway_hm_net
18                            bridge_hm_net
19                         pct_white_hm_net
20              pct_African_American_hm_net
21    pct_at_least_college_education_hm_net
22                             slope_hm_net
Name: feature, dtype: object

# Eugene 0.75 mile buffer

In [100]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,2605.056766,5732.426114,4763.753429,7757.027957,1.457986,52.194247,75.815616,0.099863,23666.969040,11.338760
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,3109.267627,6804.263398,5719.795296,3825.267224,0.000000,52.194247,75.815616,0.099863,9613.449342,5.014374
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,953.503106,2705.536217,2533.301034,1032.491639,2.081199,52.194247,75.815616,0.099863,37171.714790,7.531423
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,1200.797444,2552.268402,2245.813664,4157.566427,0.955379,52.194247,75.815616,0.099863,23999.696980,10.160410
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,7064.406343,18958.604070,18274.522960,22817.578490,1.879929,52.194247,75.815616,0.099863,57255.097000,26.109649


In [101]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.30]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'Distance.to.Retail.Area',
       'Distance.to.Retail.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Grass.Area_tfm_net', 'Water.Area_tfm_net',
       'Forest.Area_tfm_net', 'University_tfm_net', 'Primary_tfm_net',
       'Path_tfm_net', 'Cycleway_tfm_net', 'Footway_tfm_net',
       'pct_white_tfm_net', 'pct_African_American_tfm_net',
       'Student.Access_tfm_net', 'Median.Age_tfm_net',
       'Median_HH_income_tfm_net', 'employment_density_tfm_net',
       'Number.of.jobs_tfm_net', 'sep_bikeway_tfm_net', 'BikeFac_t

In [102]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw'],axis=1)

In [103]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 277 ms


,feature,VIF
3,Distance.to.Park,1.760466
10,Water.Area_tfm_net,2.040554
13,Primary_tfm_net,2.181464
11,Forest.Area_tfm_net,2.214804
15,pct_white_tfm_net,2.226913
9,Grass.Area_tfm_net,2.479424
5,Distance.to.forest,2.640330
4,Distance.to.Water.Center,2.686497
2,Distance.to.Retail.Center,2.937261
8,log_stl_raw,2.972389


In [104]:
vif_data['feature']

1                        stv_c_adb
2        Distance.to.Retail.Center
3                 Distance.to.Park
4         Distance.to.Water.Center
5               Distance.to.forest
6           min_dist_to_university
7                   log_stv_nc_adb
8                      log_stl_raw
9               Grass.Area_tfm_net
10              Water.Area_tfm_net
11             Forest.Area_tfm_net
12              University_tfm_net
13                 Primary_tfm_net
14                Cycleway_tfm_net
15               pct_white_tfm_net
16    pct_African_American_tfm_net
Name: feature, dtype: object

# Eugene 1 mile buffer

In [105]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,2505.725855,5496.670993,4627.865974,9202.383087,1.590984,52.194247,75.815616,0.099863,28544.49543,16.209625
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,3386.983811,7503.764387,6317.145651,5420.767412,0.000000,52.194247,75.815616,0.099863,21846.48920,10.026571
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,945.493393,2631.548998,2447.405465,2214.140436,2.274989,52.194247,75.815616,0.099863,72252.69708,15.894440
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,1396.973331,3025.016059,2672.488455,5838.640886,0.950457,52.194247,75.815616,0.099863,32880.55259,16.171030
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,6344.557303,16939.840420,16245.852680,34776.566620,1.807018,52.194247,75.815616,0.099863,102933.75180,45.140987


In [106]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_om_net',
       'Bike.Commuter_om_net', 'Percentage.of.Bike.Commuter_om_net',
       'bike.commuter.density_om_net', 'Grass.Area_om_net',
       'Water.Area_om_net', 'Bicycle.Parking_om_net', 'college_om_net',
       'University_om_net', 'Primary_om_net', 'Path_om_net', 'Cycleway_om_net',
       'Footway_om_net', 'p

In [108]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop([ 'stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw'],axis=1)

In [109]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 561 ms


,feature,VIF
6,log_stv_nc_adb,1.574392
4,Distance.to.Park,1.697162
1,tertiary_binary,1.708206
7,log_stl_raw,1.872599
5,Distance.to.forest,2.072940
11,Water.Area_om_net,2.381332
15,pct_white_om_net,2.636602
14,Primary_om_net,2.819934
12,college_om_net,2.893116
2,Distance.to.Industrial.Center,3.029446


In [110]:
vif_data['feature']

1                           tertiary_binary
2             Distance.to.Industrial.Center
3                   Distance.to.Retail.Area
4                          Distance.to.Park
5                        Distance.to.forest
6                            log_stv_nc_adb
7                               log_stl_raw
8                 Distance_to_Water_Body_mi
9        Percentage.of.Bike.Commuter_om_net
10                        Grass.Area_om_net
11                        Water.Area_om_net
12                           college_om_net
13                        University_om_net
14                           Primary_om_net
15                         pct_white_om_net
16    pct_at_least_college_education_om_net
Name: feature, dtype: object

# Eugene 1.5 mile buffer Test

In [111]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,2547.905368,5445.967081,4735.598443,16646.090980,2.140678,52.194247,75.815616,0.099863,41592.31486,31.157091
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,3212.790701,7397.877376,6168.447663,9073.505588,0.000000,52.194247,75.815616,0.099863,43380.68769,27.171879
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,1176.421544,3249.771321,2989.944323,8104.935998,2.400127,52.194247,75.815616,0.099863,137974.29400,36.479439
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,1535.492468,3248.818577,2865.774211,12221.373790,1.024698,52.194247,75.815616,0.099863,60210.99915,32.882896
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,5514.967645,13811.195580,13023.793390,55546.856500,1.948265,52.194247,75.815616,0.099863,230364.07390,92.258786


In [112]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.25]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_ohm_net',
       'Bike.Commuter_ohm_net', 'Percentage.of.Bike.Commuter_ohm_net',
       'bike.commuter.density_ohm_net', 'Retail.Area_ohm_net',
       'Grass.Area_ohm_net', 'Park.Area_ohm_net', 'Water.Area_ohm_net',
       'Bicycle.Parking_ohm_net', 'college_ohm_net', 'University_ohm_net',
       'Primary_ohm_net', 

In [114]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw'],axis=1)

In [115]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 619 ms


,feature,VIF
5,Distance.to.Park,1.725406
2,tertiary_binary,1.825087
16,population_density_ohm_net,2.143058
3,Distance.to.Industrial.Area,2.261635
10,log_stl_raw,2.369408
7,Distance.to.forest,2.542446
6,Distance.to.Water.Center,3.341168
13,Water.Area_ohm_net,3.848938
9,log_stv_nc_adb,3.855223
12,Park.Area_ohm_net,3.884235


In [116]:
vif_data['feature']

1                       stv_c_adb
2                 tertiary_binary
3     Distance.to.Industrial.Area
4       Distance.to.Retail.Center
5                Distance.to.Park
6        Distance.to.Water.Center
7              Distance.to.forest
8             min_dist_to_college
9                  log_stv_nc_adb
10                    log_stl_raw
11            Retail.Area_ohm_net
12              Park.Area_ohm_net
13             Water.Area_ohm_net
14             University_ohm_net
15              pct_white_ohm_net
16     population_density_ohm_net
Name: feature, dtype: object

# Eugene 2 mile buffer

In [117]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
e_data1=pd.concat([d7, d1.reindex(d7.index)], axis=1)
e_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
147,513.408379,12.575342,3.780822,985,0,0,0,0,0,1,...,2456.695908,5424.452972,4812.125974,32961.56429,2.088664,52.194247,75.815616,0.099863,86955.45017,64.614200
148,109.807738,7.780822,2.397260,203,0,0,0,0,0,1,...,3064.774876,7290.571020,6035.541489,16915.47517,0.000000,52.194247,75.815616,0.099863,133682.92850,61.216622
149,330.297326,10.383562,4.863014,2766,0,0,0,0,0,1,...,1769.579449,4474.830028,4083.044499,20034.76777,2.098662,52.194247,75.815616,0.099863,191007.29350,57.279205
150,429.732467,19.589041,7.342466,1233,0,0,0,0,0,1,...,1732.365167,3653.242783,3208.289749,29884.23225,1.183984,52.194247,75.815616,0.099863,113423.05130,64.485386
151,410.282143,6.972603,4.479452,2154,0,0,0,0,0,1,...,4829.998897,11503.817300,10655.851360,74352.75581,2.093512,52.194247,75.815616,0.099863,423183.61790,154.650904


In [118]:
corrMatrix1 = e_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.Water.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_college',
       'min_dist_to_university', 'stv_nc_adb', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_tm_net',
       'Bike.Commuter_tm_net', 'Percentage.of.Bike.Commuter_tm_net',
       'bike.commuter.density_tm_net', 'Retail.Area_tm_net',
       'Grass.Area_tm_net', 'Park.Area_tm_net', 'Water.Area_tm_net',
       'Bicycle.Parking_tm_net', 'college

In [119]:
edata0=e_data1[top_corr_features]
edata00=edata0.drop(['stv_adb','Distance.to.Grass','Distance.to.Grass.Center','stv_nc_adb','stl_raw','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary'],axis=1)

In [120]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

edata00
X_dataf = edata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = edata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 792 ms


,feature,VIF
5,Distance.to.Park.Center,1.660568
2,tertiary_binary,1.717021
16,HH_density_tm_net,2.334040
10,log_stl_raw,2.369713
7,Distance.to.Forest.Center,2.598495
11,Retail.Area_tm_net,2.659684
15,pct_female_tm_net,2.820712
14,pct_white_tm_net,3.084329
6,Distance.to.Water.Center,3.091289
3,Distance.to.Industrial.Center,3.288929


In [121]:
vif_data['feature']

1                         stv_c_adb
2                   tertiary_binary
3     Distance.to.Industrial.Center
4         Distance.to.Retail.Center
5           Distance.to.Park.Center
6          Distance.to.Water.Center
7         Distance.to.Forest.Center
8               min_dist_to_college
9                    log_stv_nc_adb
10                      log_stl_raw
11               Retail.Area_tm_net
12                   Primary_tm_net
13                  maxspeed_tm_net
14                 pct_white_tm_net
15                pct_female_tm_net
16                HH_density_tm_net
Name: feature, dtype: object

# Bend Variable Selections

In [123]:
be_data=data[data['region']=='Bend']
be_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
0,1,P001BEN,Colorado Ave.,permanent,2019,Bend,2.0,46.209524,6.561644,2.616438,...,3056.0,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341,1
1,2,P002BEN,Franklin Ave. Both Sides,permanent,2019,Bend,2.0,220.426625,6.958904,3.068493,...,3056.0,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931,2
2,3,P003BEN,Galveston Ave. Both Sides,permanent,2019,Bend,3.0,127.953968,13.726027,4.369863,...,3056.0,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863,3
3,4,P004BEN,Newport Ave. Both Sides,permanent,2019,Bend,1.0,109.378321,7.027397,2.178082,...,3056.0,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368,4
4,5,P005BEN,Portland Ave. Both Sides,permanent,2019,Bend,2.0,61.295238,5.260274,1.698630,...,3056.0,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038,5


In [124]:
# Clean the data
an_data=be_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

[]


In [125]:
be1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
be1_data=be1_data.drop(['Unnamed: 0','year','travel_pattern','latitude','longitude','rowIndex','Bike.Share.Origin_otm',
 'Bike.Share.Destination_otm',
 'Bike.Share.Crossing_otm','Bike.Share.Origin_qm',
 'Bike.Share.Destination_qm',
 'Bike.Share.Crossing_qm','Bike.Share.Origin_tfm',
 'Bike.Share.Destination_tfm',
 'Bike.Share.Crossing_tfm','Bike.Share.Origin_om',
 'Bike.Share.Destination_om',
 'Bike.Share.Crossing_om','Bike.Share.Origin_ohm',
 'Bike.Share.Destination_ohm',
 'Bike.Share.Crossing_ohm','Bike.Share.Origin_tm',
 'Bike.Share.Destination_tm',
 'Bike.Share.Crossing_tm','Bike.Share.Origin_qm_net',
 'Bike.Share.Destination_qm_net',
 'Bike.Share.Crossing_qm_net','Bike.Share.Origin_otm_net',
 'Bike.Share.Destination_otm_net',
 'Bike.Share.Crossing_otm_net','Bike.Share.Origin_hm_net',
 'Bike.Share.Destination_hm_net',
 'Bike.Share.Crossing_hm_net','Bike.Share.Origin_tfm_net',
 'Bike.Share.Destination_tfm_net',
 'Bike.Share.Crossing_tfm_net','Bike.Share.Origin_om_net',
 'Bike.Share.Destination_om_net',
 'Bike.Share.Crossing_om_net','Bike.Share.Origin_ohm_net',
 'Bike.Share.Destination_ohm_net',
 'Bike.Share.Crossing_ohm_net', 'Bike.Share.Origin_tm_net',
 'Bike.Share.Destination_tm_net',
 'Bike.Share.Crossing_tm_net','Bike.Share.best_otm',
 'Bike.Share.best_qm',
 'Bike.Share.best_hm',
 'Bike.Share.best_tfm',
 'Bike.Share.best_om',
 'Bike.Share.best_ohm',
 'Bike.Share.best_tm','Bike.Share.Origin_hm',
 'Bike.Share.Destination_hm',
 'Bike.Share.Crossing_hm','Bike.Share.route_otm_net',
'Bike.Share.route_otm','Bike.Share.route_qm','Bike.Share.route_tfm','Bike.Share.route_qm_net',
'Bike.Share.route_hm','Bike.Share.route_om','Bike.Share.route_otm_net','Bike.Share.route_qm_net','Bike.Share.route_hm_net','Bike.Share.route_tfm_net','Bike.Share.route_om_net','Bike.Share.route_ohm_net','Bike.Share.route_tm_net'],axis=1)

be1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,8.067776,10.090865,1.248328,7.709046,10.644986,28.863905,35.305839,108.195110,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,7.870166,9.944457,0.000000,0.000000,3.995612,49.828057,65.959899,230.465979,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,8.048469,10.738085,1.191490,6.758242,27.855287,31.094049,41.330493,98.318063,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,7.600402,9.683263,0.722608,8.910716,26.495791,28.563976,54.080251,118.796144,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,7.235619,9.069843,5.137481,7.813417,12.608750,31.528644,55.880389,182.273542,0.559726,0.005038


In [126]:
d1=be1_data.loc[:,~be1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,0,1,0,2.023089,1.285490,1.598419,8.067776,10.090865,0.512552,0.113341
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,0,0,0,2.074291,1.403273,1.587276,7.870166,9.944457,0.282730,0.426931
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,0,0,0,2.689617,1.680802,2.337582,8.048469,10.738085,0.499890,0.010863
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,0,0,0,2.082860,1.156278,1.766325,7.600402,9.683263,0.344087,0.010368
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,0,0,1,1.834224,0.992744,1.517683,7.235619,9.069843,0.559726,0.005038


In [127]:
net_data=be1_data.loc[:,be1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
0,35.531170,0.678076,1.908397,24.365364,503.063397,0.0,0.0,339.145348,0.0,10949.72823,...,35001.51393,60971.18678,116621.72730,0.395530,1.440465,3.565952,9.355694,15.616797,33.203417,57.739067
1,39.005450,1.704244,4.369246,55.724012,326.972002,0.0,0.0,2609.755647,0.0,0.00000,...,21772.75413,52411.79064,112209.70950,0.336709,1.189527,2.748207,7.121148,12.788902,27.702900,58.699022
2,45.759783,2.886810,6.308618,128.518935,489.712962,0.0,0.0,2489.827234,0.0,0.00000,...,25354.47209,67283.59369,109300.85250,0.069645,0.311978,2.213339,5.935694,12.814490,29.903869,54.308968
3,42.307763,0.927522,2.192320,35.713162,346.534723,0.0,0.0,1274.619412,0.0,0.00000,...,20762.70934,47873.90545,105821.84770,0.081332,0.577948,3.120323,7.120087,13.484632,29.851699,57.632743
4,21.726393,1.662712,7.652959,82.319830,346.565625,0.0,0.0,196.668745,0.0,0.00000,...,14571.25813,46345.18729,89953.08027,0.278187,1.237188,3.608611,7.176205,12.228374,31.115168,53.971717


In [128]:
be1_data=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
be1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,35001.51393,60971.18678,116621.72730,0.395530,1.440465,3.565952,9.355694,15.616797,33.203417,57.739067
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,21772.75413,52411.79064,112209.70950,0.336709,1.189527,2.748207,7.121148,12.788902,27.702900,58.699022
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,25354.47209,67283.59369,109300.85250,0.069645,0.311978,2.213339,5.935694,12.814490,29.903869,54.308968
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,20762.70934,47873.90545,105821.84770,0.081332,0.577948,3.120323,7.120087,13.484632,29.851699,57.632743
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,14571.25813,46345.18729,89953.08027,0.278187,1.237188,3.608611,7.176205,12.228374,31.115168,53.971717


In [130]:
corrMatrix1 = be1_data.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Industrial.Area_otm_net',
       'Park.Area_otm_net', 'Water.Area_otm_net', 'Path_otm_net',
       'Cycleway_otm_net', 'bridge_otm_net', 'Student.Access_otm_net',
       'pct_at_least_college_education_otm_net', 'Park.Area_qm_net',
       'Bicycle.Parking_qm_ne

In [132]:
bedata0=be1_data[top_corr_features]
bedata00=bedata0.drop(['stv_c_adb'],axis=1)

In [133]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.56 s


,feature,VIF
6,Industrial.Area_otm_net,1.474818
19,bridge_om_net,2.162020
11,Park.Area_qm_net,2.190179
16,School_hm_net,2.392478
10,pct_at_least_college_education_otm_net,2.413412
20,pct_African_American_om_net,2.481900
5,Distance_to_Water_Body_mi,2.613712
9,Student.Access_otm_net,2.779998
17,bridge_hm_net,2.813803
22,Percentage.of.Bike.Commuter_ohm_net,2.833056


In [134]:
vif_data['feature']

1                                 stv_nc_adb
2                        sep_onstreet_binary
3                             BikeFac_binary
4                              log_stv_c_adb
5                  Distance_to_Water_Body_mi
6                    Industrial.Area_otm_net
7                         Water.Area_otm_net
8                           Cycleway_otm_net
9                     Student.Access_otm_net
10    pct_at_least_college_education_otm_net
11                          Park.Area_qm_net
12                    Bicycle.Parking_qm_net
13                             bridge_qm_net
14     pct_at_least_college_education_qm_net
15                    Bicycle.Parking_hm_net
16                             School_hm_net
17                             bridge_hm_net
18                        Grass.Area_tfm_net
19                             bridge_om_net
20               pct_African_American_om_net
21                         pct_female_om_net
22       Percentage.of.Bike.Commuter_ohm_net
23        

# Bend 0.1 mile buffer

In [135]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1092.786565,2091.766480,1919.990666,302.954352,1.742248,48.478082,61.352877,0.028959,195.782541,0.395530
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1084.678535,2295.180081,1901.475198,200.280232,2.408299,48.478082,61.352877,0.028959,889.554467,0.336709
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1710.957460,3001.449800,2713.463315,25.678051,1.097476,48.478082,61.352877,0.028959,367.727918,0.069645
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1116.586613,2383.468959,1924.100868,121.718182,2.485532,48.478082,61.352877,0.028959,429.433108,0.081332
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,905.948245,2151.814862,1716.729840,222.945515,0.000000,48.478082,61.352877,0.028959,691.451073,0.278187


In [149]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Industrial.Area_otm_net',
       'Park.Area_otm_net', 'Water.Area_otm_net', 'Path_otm_net',
       'Cycleway_otm_net', 'bridge_otm_net', 'Student.Access_otm_net',
       'pct_at_least_college_education_otm_net', 'sep_bikeway_otm_net'],
      dtype='object')

In [150]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','stv_adb'],axis=1)

In [151]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 263 ms


,feature,VIF
7,Industrial.Area_otm_net,1.163913
2,stv_nc_adb,1.592517
12,Student.Access_otm_net,1.696832
9,Water.Area_otm_net,2.031610
6,Distance_to_Water_Body_mi,2.054721
1,min_dist_to_college,2.092415
4,BikeFac_binary,2.143660
8,Park.Area_otm_net,2.218868
13,pct_at_least_college_education_otm_net,2.311934
5,log_stl_raw,2.485897


In [152]:
vif_data['feature']

1                        min_dist_to_college
2                                 stv_nc_adb
3                        sep_onstreet_binary
4                             BikeFac_binary
5                                log_stl_raw
6                  Distance_to_Water_Body_mi
7                    Industrial.Area_otm_net
8                          Park.Area_otm_net
9                         Water.Area_otm_net
10                          Cycleway_otm_net
11                            bridge_otm_net
12                    Student.Access_otm_net
13    pct_at_least_college_education_otm_net
Name: feature, dtype: object

# 0.25 mile buffer

In [153]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.836151,2036.203607,1862.396701,829.005006,1.856461,48.478082,61.352877,0.028959,3168.931840,1.440465
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1084.679453,2295.157044,1901.477295,1351.149336,1.720310,48.478082,61.352877,0.028959,924.377060,1.189527
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1503.180649,2744.512703,2440.557228,387.590203,1.412683,48.478082,61.352877,0.028959,1647.243197,0.311978
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1249.236493,2808.364961,2215.072908,1496.004378,1.812751,48.478082,61.352877,0.028959,1111.749342,0.577948
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,902.528117,2145.488551,1717.049191,941.551494,0.000000,48.478082,61.352877,0.028959,1428.936640,1.237188


In [154]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Park.Area_qm_net',
       'Bicycle.Parking_qm_net', 'Cycleway_qm_net', 'bridge_qm_net',
       'pct_at_least_college_education_qm_net', 'sep_bikeway_qm_net'],
      dtype='object')

In [156]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [157]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 246 ms


,feature,VIF
8,Park.Area_qm_net,1.392765
9,Bicycle.Parking_qm_net,1.446351
7,Distance_to_Water_Body_mi,1.536331
11,bridge_qm_net,1.811379
10,Cycleway_qm_net,1.911369
4,BikeFac_binary,2.044325
12,pct_at_least_college_education_qm_net,2.156764
3,sep_onstreet_binary,2.646930
1,min_dist_to_college,3.239889
6,log_stl_raw,3.345506


In [158]:
vif_data['feature']

1                       min_dist_to_college
2                                stv_nc_adb
3                       sep_onstreet_binary
4                            BikeFac_binary
5                             log_stv_c_adb
6                               log_stl_raw
7                 Distance_to_Water_Body_mi
8                          Park.Area_qm_net
9                    Bicycle.Parking_qm_net
10                          Cycleway_qm_net
11                            bridge_qm_net
12    pct_at_least_college_education_qm_net
Name: feature, dtype: object

# Bend 0.5 mile buffer

In [160]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.075347,2029.870332,1849.840318,3179.167538,1.796814,48.478082,61.352877,0.028959,8226.788063,3.565952
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1140.645925,2363.272923,1975.098575,6298.912469,1.717111,48.478082,61.352877,0.028959,2418.948981,2.748207
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1316.888252,2592.371997,2251.099836,1822.446713,1.598114,48.478082,61.352877,0.028959,8156.981385,2.213339
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1273.780756,2877.745161,2276.266710,5575.592943,1.996023,48.478082,61.352877,0.028959,6657.914592,3.120323
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1004.362315,2397.361344,1920.692370,4137.955371,0.000000,48.478082,61.352877,0.028959,4029.161672,3.608611


In [161]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Water.Area_hm_net',
       'Bicycle.Parking_hm_net', 'School_hm_net', 'bridge_hm_net',
       'pct_at_least_college_education_hm_net'],
      dtype='object')

In [163]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [164]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 223 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
10,School_hm_net,1.324243
11,bridge_hm_net,1.763557
7,Distance_to_Water_Body_mi,2.100029
4,BikeFac_binary,2.111057
3,sep_onstreet_binary,2.162831
12,pct_at_least_college_education_hm_net,2.623334
1,min_dist_to_college,3.236097
8,Water.Area_hm_net,3.628728
9,Bicycle.Parking_hm_net,3.997370
2,stv_nc_adb,4.094290


In [165]:
vif_data['feature']

1                       min_dist_to_college
2                                stv_nc_adb
3                       sep_onstreet_binary
4                            BikeFac_binary
5                             log_stv_c_adb
6                               log_stl_raw
7                 Distance_to_Water_Body_mi
8                         Water.Area_hm_net
9                    Bicycle.Parking_hm_net
10                            School_hm_net
11                            bridge_hm_net
12    pct_at_least_college_education_hm_net
Name: feature, dtype: object

# 0.75 mile buffer

In [166]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1023.544330,1999.166320,1801.170916,8961.622695,1.718788,48.478082,61.352877,0.028959,24512.96424,9.355694
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1185.968034,2434.314963,2041.267896,9175.566610,1.809894,48.478082,61.352877,0.028959,11881.27513,7.121148
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1217.786570,2640.446631,2202.291352,5881.091374,1.820654,48.478082,61.352877,0.028959,12950.78401,5.935694
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1262.446464,2830.839076,2276.899147,7704.876164,2.141138,48.478082,61.352877,0.028959,11469.32877,7.120087
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1098.051723,2586.992843,2077.994523,8422.905274,0.000000,48.478082,61.352877,0.028959,7652.72686,7.176205


In [167]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features
#plt.figure(figsize=(18,14))
#plt.title("2019_cor-relation>0.20 with AADBT")
#g = sns.heatmap(be1_data11[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Grass.Area_tfm_net',
       'Bicycle.Parking_tfm_net', 'School_tfm_net', 'maxspeed_tfm_net',
       'pct_at_least_college_education_tfm_net'],
      dtype='object')

In [169]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [170]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 238 ms


,feature,VIF
6,Distance_to_Water_Body_mi,1.881115
3,sep_onstreet_binary,1.893750
4,BikeFac_binary,1.906242
1,stv_adb,2.158343
7,Grass.Area_tfm_net,2.161550
2,Distance.to.Park,2.412180
9,School_tfm_net,2.852527
8,Bicycle.Parking_tfm_net,2.853867
10,pct_at_least_college_education_tfm_net,3.097758
5,log_stv_stl,3.275002


In [171]:
vif_data['feature']

1                                    stv_adb
2                           Distance.to.Park
3                        sep_onstreet_binary
4                             BikeFac_binary
5                                log_stv_stl
6                  Distance_to_Water_Body_mi
7                         Grass.Area_tfm_net
8                    Bicycle.Parking_tfm_net
9                             School_tfm_net
10    pct_at_least_college_education_tfm_net
Name: feature, dtype: object

# Bend 1.0 mile buffer

In [172]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,987.765909,2039.114372,1773.560854,14210.18096,1.807770,48.478082,61.352877,0.028959,35001.51393,15.616797
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1174.910427,2479.059314,2071.134552,12359.63894,1.742249,48.478082,61.352877,0.028959,21772.75413,12.788902
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,1107.086579,2486.103335,2043.481168,11142.91564,2.240183,48.478082,61.352877,0.028959,25354.47209,12.814490
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1193.187839,2686.377224,2187.958431,11303.31561,2.256581,48.478082,61.352877,0.028959,20762.70934,13.484632
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1152.249511,2637.286420,2140.344673,11169.65567,0.000000,48.478082,61.352877,0.028959,14571.25813,12.228374


In [173]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Grass.Area_om_net',
       'Bicycle.Parking_om_net', 'School_om_net', 'maxspeed_om_net',
       'bridge_om_net', 'pct_African_American_om_net', 'pct_male_om_net',
       'pct_female_om_net', 'pct_at_least_college_education_om_net',
       'Number.of.jobs_om_net

In [175]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Commercial.Area.Center'],axis=1)

In [176]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 330 ms


,feature,VIF
6,Distance_to_Water_Body_mi,1.427648
9,bridge_om_net,1.821716
3,sep_onstreet_binary,1.828015
10,pct_African_American_om_net,1.849771
4,BikeFac_binary,1.881103
7,Grass.Area_om_net,2.615668
11,pct_female_om_net,2.617448
1,min_dist_to_college,2.620145
5,log_stv_c_adb,2.632273
8,School_om_net,2.752625


In [177]:
vif_data['feature']

1             min_dist_to_college
2                      stv_nc_adb
3             sep_onstreet_binary
4                  BikeFac_binary
5                   log_stv_c_adb
6       Distance_to_Water_Body_mi
7               Grass.Area_om_net
8                   School_om_net
9                   bridge_om_net
10    pct_African_American_om_net
11              pct_female_om_net
12          Number.of.jobs_om_net
Name: feature, dtype: object

# bend 1.5 mile buffer

In [178]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1056.789068,2430.079342,1990.766003,20294.00738,2.193156,48.478082,61.352877,0.028959,60971.18678,33.203417
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1179.750560,2658.042382,2181.243968,21062.80977,2.041506,48.478082,61.352877,0.028959,52411.79064,27.702900
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,939.174351,2153.735239,1762.012857,19474.65755,2.427138,48.478082,61.352877,0.028959,67283.59369,29.903869
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1085.433314,2467.190081,2013.320626,19438.67456,2.233351,48.478082,61.352877,0.028959,47873.90545,29.851699
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1212.423151,2759.914670,2242.972630,17313.71673,0.000000,48.478082,61.352877,0.028959,46345.18729,31.115168


In [179]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Percentage.of.Bike.Commuter_ohm_net',
       'Grass.Area_ohm_net', 'University_ohm_net', 'maxspeed_ohm_net',
       'pct_African_American_ohm_net',
       'pct_at_least_college_education_ohm_net', 'Median.Age_ohm_net'],
      dtype='object')

In [181]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [182]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 257 ms


,feature,VIF
6,Distance_to_Water_Body_mi,1.419360
9,University_ohm_net,1.610743
3,sep_onstreet_binary,1.819441
7,Percentage.of.Bike.Commuter_ohm_net,1.851580
8,Grass.Area_ohm_net,1.858478
1,Distance.to.Park,1.912908
4,BikeFac_binary,1.976804
10,pct_African_American_ohm_net,2.428752
11,pct_at_least_college_education_ohm_net,3.079881
5,log_stv_c_adb,3.200300


In [183]:
vif_data['feature']

1                           Distance.to.Park
2                                 stv_nc_adb
3                        sep_onstreet_binary
4                             BikeFac_binary
5                              log_stv_c_adb
6                  Distance_to_Water_Body_mi
7        Percentage.of.Bike.Commuter_ohm_net
8                         Grass.Area_ohm_net
9                         University_ohm_net
10              pct_African_American_ohm_net
11    pct_at_least_college_education_ohm_net
Name: feature, dtype: object

# Bend 2.0 mile buffer

In [184]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
be_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
be_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
0,46.209524,6.561644,2.616438,3189,0,1,0,0,0,0,...,1026.478817,2415.698097,1941.406570,26072.11710,2.264271,48.478082,61.352877,0.028959,116621.72730,57.739067
1,220.426625,6.958904,3.068493,2617,0,1,0,0,0,0,...,1179.759608,2740.534260,2212.566542,26705.89684,2.200950,48.478082,61.352877,0.028959,112209.70950,58.699022
2,127.953968,13.726027,4.369863,3128,0,1,0,0,0,0,...,942.623989,2212.217243,1789.363360,25163.68821,2.390589,48.478082,61.352877,0.028959,109300.85250,54.308968
3,109.378321,7.027397,2.178082,1998,0,1,0,0,0,0,...,1082.118678,2504.794925,2029.680644,26006.37448,2.361176,48.478082,61.352877,0.028959,105821.84770,57.632743
4,61.295238,5.260274,1.698630,1387,0,0,1,0,0,0,...,1170.006244,2688.548616,2183.688248,25701.46188,0.000000,48.478082,61.352877,0.028959,89953.08027,53.971717


In [185]:
corrMatrix1 = be_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'cycleway_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb',
       'log_stl_raw', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Percentage.of.Bike.Commuter_tm_net',
       'maxspeed_tm_net', 'Median.Age_tm_net'],
      dtype='object')

In [186]:
bedata0=be_data11[top_corr_features]
bedata00=bedata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center'],axis=1)

In [187]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bedata00
X_dataf = bedata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bedata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 214 ms


,feature,VIF
6,Distance_to_Water_Body_mi,1.308050
2,stv_nc_adb,1.391082
7,Percentage.of.Bike.Commuter_tm_net,1.458677
4,BikeFac_binary,1.799693
3,sep_onstreet_binary,1.880251
1,Distance.to.Park,1.951656
5,log_stv_stl,2.154064


In [188]:
vif_data['feature']

1                      Distance.to.Park
2                            stv_nc_adb
3                   sep_onstreet_binary
4                        BikeFac_binary
5                           log_stv_stl
6             Distance_to_Water_Body_mi
7    Percentage.of.Bike.Commuter_tm_net
Name: feature, dtype: object

# Dallas all Buffers

In [189]:
da_data=data[data['region']=='Dallas']
da_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
116,117,P001DAL,AT&T Trail at Trinity Forest Trail,permanent,2019,Dallas,3.0,11.805952,0.986301,0.013699,...,NaN,0.000000,0.000000,0.000000,25.512511,43.453987,150.472751,6.966430,0.025024,117
117,118,P002DAL,Coombs Creek Trail,permanent,2019,Dallas,3.0,17.944163,5.794521,1.273973,...,NaN,0.217983,2.214549,2.278466,15.979778,59.050920,140.723741,2.805968,0.501749,118
118,119,P003DAL,Cottonwood Creek Trail at Hamilton Park,permanent,2019,Dallas,3.0,63.346667,5.726027,1.383562,...,NaN,16.780093,42.889590,56.251345,72.315602,105.249238,172.150190,10.005480,0.545815,119
119,120,P004DAL,Crawford Memorial Park Loop Trail,permanent,2019,Dallas,2.0,43.761310,0.150685,0.000000,...,NaN,40.133036,197.745903,509.494083,526.322986,559.045079,611.094139,8.688990,1.622354,120
120,121,P006DAL,Gateway Park Loop Trail,permanent,2019,Dallas,3.0,26.274603,0.000000,0.000000,...,NaN,12.408895,62.431443,105.934496,127.755741,134.067319,204.293522,5.762422,0.452393,121


In [190]:
# Clean the data
an_data=da_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Origin_hm_net', 'Bike

In [192]:
da1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
da1_data=da1_data.drop(['travel_pattern','Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
da1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,3.258097,3.944371,0.000000,0.000000,0.000000,25.512511,43.453987,150.472751,6.966430,0.025024
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,2.564949,4.481066,0.217983,2.214549,2.278466,15.979778,59.050920,140.723741,2.805968,0.501749
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,5.398163,7.304147,16.780093,42.889590,56.251345,72.315602,105.249238,172.150190,10.005480,0.545815
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,4.867534,5.007892,40.133036,197.745903,509.494083,526.322986,559.045079,611.094139,8.688990,1.622354
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,2.197225,2.197225,12.408895,62.431443,105.934496,127.755741,134.067319,204.293522,5.762422,0.452393


In [201]:
d1=da1_data.loc[:,~da1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,0,0,0,0.686274,0.013606,0.679354,3.258097,3.944371,6.966430,0.025024
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,0,0,0,1.916116,0.821528,1.708477,2.564949,4.481066,2.805968,0.501749
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,0,0,0,1.905985,0.868596,1.675687,5.398163,7.304147,10.005480,0.545815
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,0,0,0,0.140357,0.000000,0.140357,4.867534,5.007892,8.688990,1.622354
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,2.197225,2.197225,5.762422,0.452393


In [202]:
net_data=da1_data.loc[:,da1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,...,5659.389855,5670.247897,9686.580634,0.000000,0.000000,0.131572,0.131572,1.071854,1.073911,1.834580
117,34.409753,0.000000,0.000000,0.000000,250.671550,0.000000,0.0,746.876197,0.00000,0.00000,...,38927.826670,59456.060190,91244.554250,0.501005,1.027430,4.044068,7.839335,10.482349,14.370272,21.403174
118,32.562872,0.093736,0.287861,5.738817,489.787194,8880.104824,0.0,14708.449630,17894.03998,35819.19783,...,7760.490499,23065.339500,49504.598200,0.150146,0.150146,0.260083,0.632442,1.952230,5.326441,11.780089
119,7.824497,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
120,41.540610,0.000000,0.000000,0.000000,207.039417,0.000000,0.0,1749.560577,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [203]:
da_data1=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
da_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,5659.389855,5670.247897,9686.580634,0.000000,0.000000,0.131572,0.131572,1.071854,1.073911,1.834580
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,38927.826670,59456.060190,91244.554250,0.501005,1.027430,4.044068,7.839335,10.482349,14.370272,21.403174
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,7760.490499,23065.339500,49504.598200,0.150146,0.150146,0.260083,0.632442,1.952230,5.326441,11.780089
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [204]:
corrMatrix1 = da_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.50]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'stv_nc_adb',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'pct_at_least_college_education_otm_net',
       'pct_white_qm_net', 'pct_at_least_college_education_qm_net',
       'pct_white_hm_net', 'pct_African_American_hm_net',
       'pct_at_least_college_education_hm_net', 'pct_white_tfm_net',
       'pct_African_American_tfm_net',
       'pct_at_least_college_education_tfm_net', 'pct_white_om_net',
       'pct_African_American_om_net', 'pct_male_om_net', 'pct_female_om_net',
       'pct_at_least_college_education_om_net', 'pct_white_ohm_net',
       'pct_male_ohm_net', 'pct_female_ohm_net',
       'pct_at_least_college_education_ohm_net', 'Median_HH_income_ohm_net',
       'pct_white_tm_net', 'pct_male_tm_net', 'pct_female_tm_net',
       'pct_at_least_college_

In [208]:
dadata0=da_data1[top_corr_features]
dadata00=dadata0.drop(['stl_raw', 'Distance.to.Grass.Center'],axis=1)

In [209]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 241 ms


,feature,VIF
5,pct_at_least_college_education_otm_net,1.693689
2,Distance.to.Grass,1.756692
7,pct_female_om_net,1.789113
3,stv_nc_adb,2.056531
8,Median_HH_income_ohm_net,2.071705
6,pct_African_American_om_net,2.259581
1,Distance.to.Industrial.Area,2.359545
4,log_stl_raw,2.466862


In [210]:
vif_data['feature']

1               Distance.to.Industrial.Area
2                         Distance.to.Grass
3                                stv_nc_adb
4                               log_stl_raw
5    pct_at_least_college_education_otm_net
6               pct_African_American_om_net
7                         pct_female_om_net
8                  Median_HH_income_ohm_net
Name: feature, dtype: object

# Dallas 0.1 mile buffer

In [211]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,67.624384,62.316438,0.106521,0.000000,0.000000
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1751.368649,3754.204462,2196.443274,0.777186,4.674872,67.624384,62.316438,0.106521,788.268234,0.501005
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,1447.948598,3291.173077,2027.632706,424.099520,2.412495,67.624384,62.316438,0.106521,792.769607,0.150146
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,540.891219,2393.077314,747.582804,0.000000,0.712253,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,2144.404370,6761.444468,2866.847902,2.110347,2.123432,67.624384,62.316438,0.106521,0.000000,0.000000


In [212]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Bike.Commuter_otm_net',
       'bike.commuter.density_otm_net', 'Commercial.Area_otm_net',
       'Residential_Road_otm_net', 'Grass.Area_otm_net', 'Water.Area_otm_net',
       'Tertiary_otm_net', 'Path_otm_net', 'pct_white_otm_net',
      

In [213]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','Distance.to.Grass','tertiary_bike_lane_binary'],axis=1)

In [214]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 333 ms


,feature,VIF
3,Distance.to.Forest.Center,1.511032
6,tertiary_no_bike_lane_binary,1.702367
9,bike.commuter.density_otm_net,1.706686
10,Commercial.Area_otm_net,2.100522
8,Distance_to_Water_Body_mi,2.239341
2,Distance.to.Park,2.274936
15,slope_otm_net,2.279410
13,pct_white_otm_net,2.705838
5,BikeFac_binary,2.712764
11,Residential_Road_otm_net,2.767986


In [215]:
vif_data['feature']

1          Distance.to.Grass.Center
2                  Distance.to.Park
3         Distance.to.Forest.Center
4                        stv_nc_adb
5                    BikeFac_binary
6      tertiary_no_bike_lane_binary
7                       log_stl_raw
8         Distance_to_Water_Body_mi
9     bike.commuter.density_otm_net
10          Commercial.Area_otm_net
11         Residential_Road_otm_net
12                 Tertiary_otm_net
13                pct_white_otm_net
14     pct_African_American_otm_net
15                    slope_otm_net
16                  BikeFac_otm_net
Name: feature, dtype: object

# Dallas 0.25 mile buffer

In [216]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,67.624384,62.316438,0.106521,0.000000,0.000000
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1599.483688,3372.163818,2036.921387,11.499904,3.786247,67.624384,62.316438,0.106521,1619.724558,1.027430
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2364.973356,4282.670923,2791.809773,1949.446998,1.574238,67.624384,62.316438,0.106521,792.769607,0.150146
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,786.702136,3077.315375,1159.443421,0.037319,1.616979,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1790.466708,5738.577976,2492.267343,4.246990,2.863416,67.624384,62.316438,0.106521,0.000000,0.000000


In [217]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Commercial.Area_qm_net',
       'Grass.Area_qm_net', 'Water.Area_qm_net', 'Tertiary_qm_net',
       'Path_qm_net', 'pct_white_qm_net', 'pct_African_American_qm_net',
       'pct_female_qm_net', 'pct_at_least_college_education_qm_net',
     

In [225]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_adb','stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [226]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

Wall time: 190 ms


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,VIF
7,Distance.to.Forest.Center,1.459201
1,tertiary_binary,1.955483
9,BikeFac_binary,2.015873
17,slope_qm_net,2.124012
4,Distance.to.Residential.Center,2.156682
6,Distance.to.Water.Center,2.178630
8,stv_nc_adb,2.263990
15,pct_African_American_qm_net,2.290879
13,Tertiary_qm_net,2.386543
16,pct_at_least_college_education_qm_net,2.489172


In [227]:
vif_data['feature']

1                           tertiary_binary
2                               path_binary
3        Distance.to.Commercial.Area.Center
4            Distance.to.Residential.Center
5                          Distance.to.Park
6                  Distance.to.Water.Center
7                 Distance.to.Forest.Center
8                                stv_nc_adb
9                            BikeFac_binary
10                   Commercial.Area_qm_net
11                        Grass.Area_qm_net
12                        Water.Area_qm_net
13                          Tertiary_qm_net
14                              Path_qm_net
15              pct_African_American_qm_net
16    pct_at_least_college_education_qm_net
17                             slope_qm_net
Name: feature, dtype: object

# 0.5 mile buffer

In [228]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,83.827377,220.673679,72.316785,0.000000,0.345026,67.624384,62.316438,0.106521,694.702222,0.131572
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1479.253217,3187.992866,1940.868177,240.270900,3.136130,67.624384,62.316438,0.106521,12299.842630,4.044068
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2508.424661,4437.435195,2911.135226,8777.588720,1.373845,67.624384,62.316438,0.106521,1373.239665,0.260083
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,941.971176,3509.521511,1419.599492,0.124709,1.430061,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1565.668800,5059.444688,2180.562516,24.535645,2.490479,67.624384,62.316438,0.106521,0.000000,0.000000


In [229]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Intersection_Density_hm_net',
       'Commercial.Area_hm_net', 'Residential_Road_hm_net',
       'Retail.Area_hm_net', 'Grass.Area_hm_net', 'Primary_hm_net',
       'Secondary_hm_net', 'Tertiary_hm_net', 'Path_hm_net',
       'pct_white_hm_

In [230]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [231]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 297 ms


,feature,VIF
4,Distance.to.Forest.Center,1.276892
6,BikeFac_binary,1.505219
1,Distance.to.Residential.Area,1.881655
9,Residential_Road_hm_net,1.961102
5,stv_nc_adb,2.078606
2,Distance.to.Park,2.415270
12,Primary_hm_net,2.455394
3,Distance.to.Water.Center,2.457366
13,Secondary_hm_net,2.841112
8,Intersection_Density_hm_net,2.931475


In [232]:
vif_data['feature']

1              Distance.to.Residential.Area
2                          Distance.to.Park
3                  Distance.to.Water.Center
4                 Distance.to.Forest.Center
5                                stv_nc_adb
6                            BikeFac_binary
7                               log_stl_raw
8               Intersection_Density_hm_net
9                   Residential_Road_hm_net
10                       Retail.Area_hm_net
11                        Grass.Area_hm_net
12                           Primary_hm_net
13                         Secondary_hm_net
14                              Path_hm_net
15    pct_at_least_college_education_hm_net
16                  Median_HH_income_hm_net
Name: feature, dtype: object

# Dallas 0.75 mile buffer

In [233]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,84.264046,221.578154,72.893853,9.544867,0.352959,67.624384,62.316438,0.106521,694.702222,0.131572
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1431.654326,3181.026064,1929.639239,641.370490,3.271300,67.624384,62.316438,0.106521,25437.395660,7.839335
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2444.422445,4816.715421,2907.944583,23876.575380,2.004750,67.624384,62.316438,0.106521,3339.295437,0.632442
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,1041.812892,3781.425072,1589.103547,29.206409,1.910388,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1407.052857,4720.643334,1941.339150,214.483125,1.954522,67.624384,62.316438,0.106521,0.000000,0.000000


In [234]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Intersection_Density_tfm_net',
       'Commercial.Area_tfm_net', 'Industrial.Area_tfm_net',
       'Residential_Road_tfm_net', 'Grass.Area_tfm_net', 'Secondary_tfm_net',
       'Tertiary_tfm_net', 'Path_tfm_net', 'maxspeed_tfm_net',
       

In [236]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [237]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 265 ms


,feature,VIF
6,BikeFac_binary,1.438860
9,Industrial.Area_tfm_net,1.757256
4,Distance.to.Forest.Center,2.014751
12,maxspeed_tfm_net,2.046528
1,Distance.to.Commercial.Area.Center,2.406233
10,Residential_Road_tfm_net,2.434579
11,Tertiary_tfm_net,2.708441
13,pct_African_American_tfm_net,2.732899
5,stv_nc_adb,2.753730
7,Distance_to_Water_Body_mi,2.756329


In [238]:
vif_data['feature']

1         Distance.to.Commercial.Area.Center
2             Distance.to.Residential.Center
3                           Distance.to.Park
4                  Distance.to.Forest.Center
5                                 stv_nc_adb
6                             BikeFac_binary
7                  Distance_to_Water_Body_mi
8               Intersection_Density_tfm_net
9                    Industrial.Area_tfm_net
10                  Residential_Road_tfm_net
11                          Tertiary_tfm_net
12                          maxspeed_tfm_net
13              pct_African_American_tfm_net
14    pct_at_least_college_education_tfm_net
15                  Median_HH_income_tfm_net
16                             slope_tfm_net
Name: feature, dtype: object

# Dallas 1 mile buffer

In [239]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,89.406725,232.230215,79.690016,65.085381,0.313574,67.624384,62.316438,0.106521,5659.389855,1.071854
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1576.917484,3614.882727,2146.905982,1514.844246,2.993773,67.624384,62.316438,0.106521,38927.826670,10.482349
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2510.014165,5142.960966,2986.527390,29361.234390,2.100825,67.624384,62.316438,0.106521,7760.490499,1.952230
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,1069.209840,3847.894456,1629.949629,72.862059,1.721329,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1370.387805,4856.377519,1980.910125,351.422567,1.908667,67.624384,62.316438,0.106521,0.000000,0.000000


In [240]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Intersection_Density_om_net',
       'Commercial.Area_om_net', 'Industrial.Area_om_net',
       'Residential_Road_om_net', 'Grass.Area_om_net', 'Secondary_om_net',
       'Tertiary_om_net', 'Path_om_net', 'maxspeed_om_net', 'pct_white_om_ne

In [242]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary','Distance.to.Residential.Center','tertiary_binary'],axis=1)

In [243]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 235 ms


,feature,VIF
7,BikeFac_binary,1.575518
5,Distance.to.Forest.Center,1.654264
2,Distance.to.Residential.Area,1.816300
4,Distance.to.Water.Center,1.975450
9,Industrial.Area_om_net,2.088224
6,stv_nc_adb,2.172606
12,maxspeed_om_net,2.607392
3,Distance.to.Park,2.903772
8,log_stl_raw,2.990086
10,Residential_Road_om_net,3.252665


In [244]:
vif_data['feature']

1     Distance.to.Commercial.Area.Center
2           Distance.to.Residential.Area
3                       Distance.to.Park
4               Distance.to.Water.Center
5              Distance.to.Forest.Center
6                             stv_nc_adb
7                         BikeFac_binary
8                            log_stl_raw
9                 Industrial.Area_om_net
10               Residential_Road_om_net
11                           Path_om_net
12                       maxspeed_om_net
13                      pct_white_om_net
14               Median_HH_income_om_net
15                          slope_om_net
Name: feature, dtype: object

# Dallas 1.50 mile buffer

In [245]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,84.002720,221.036868,72.548505,135.375996,0.291810,67.624384,62.316438,0.106521,5670.247897,1.073911
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1753.415896,4047.020474,2376.654632,10990.698470,2.204392,67.624384,62.316438,0.106521,59456.060190,14.370272
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2083.586212,5258.399955,2840.740317,35335.596940,2.167944,67.624384,62.316438,0.106521,23065.339500,5.326441
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,1140.364363,4283.969932,1758.291216,358.898523,1.313075,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1583.719602,5834.255172,2361.609393,1486.627892,1.630688,67.624384,62.316438,0.106521,0.000000,0.000000


In [246]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Commercial.Area_ohm_net',
       'Industrial.Area_ohm_net', 'Residential_Road_ohm_net',
       'Retail.Area_ohm_net', 'Grass.Area_ohm_net', 'School_ohm_net',
       'Secondary_ohm_net', 'Tertiary_ohm_net', 'cycleway_lane_all_ohm_net',
     

In [248]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [249]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 284 ms


,feature,VIF
7,BikeFac_binary,1.414049
2,Distance.to.Residential.Area,1.848948
3,Distance.to.Park,2.128723
5,Distance.to.forest,2.191758
6,stv_nc_adb,2.234063
14,pct_white_ohm_net,2.323062
10,Retail.Area_ohm_net,2.392001
9,Industrial.Area_ohm_net,2.638673
4,Distance.to.Water.Center,2.788279
13,cycleway_lane_all_ohm_net,2.883337


In [250]:
vif_data['feature']

1     Distance.to.Industrial.Center
2      Distance.to.Residential.Area
3                  Distance.to.Park
4          Distance.to.Water.Center
5                Distance.to.forest
6                        stv_nc_adb
7                    BikeFac_binary
8                       log_stl_raw
9           Industrial.Area_ohm_net
10              Retail.Area_ohm_net
11               Grass.Area_ohm_net
12                   School_ohm_net
13        cycleway_lane_all_ohm_net
14                pct_white_ohm_net
15                    slope_ohm_net
Name: feature, dtype: object

# Dallas 2 miles buffer

In [251]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
da_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
da_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
116,11.805952,0.986301,0.013699,25,0,0,0,0,0,1,...,129.981215,340.578619,119.526953,421.332561,0.994653,67.624384,62.316438,0.106521,9686.580634,1.834580
117,17.944163,5.794521,1.273973,12,0,0,0,0,1,0,...,1874.018892,5006.008852,2663.096953,21857.928880,1.852091,67.624384,62.316438,0.106521,91244.554250,21.403174
118,63.346667,5.726027,1.383562,220,0,0,0,0,0,0,...,2401.598888,6118.937381,3323.646443,52737.180250,2.127659,67.624384,62.316438,0.106521,49504.598200,11.780089
119,43.761310,0.150685,0.000000,129,0,0,0,0,0,0,...,1144.572266,4429.708077,1751.826993,939.475111,1.119172,67.624384,62.316438,0.106521,0.000000,0.000000
120,26.274603,0.000000,0.000000,8,0,0,0,0,0,0,...,1621.451614,5817.279085,2322.976166,2644.124866,1.354728,67.624384,62.316438,0.106521,0.000000,0.000000


In [252]:
corrMatrix1 = da_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'path_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Industrial.Area',
       'Distance.to.Industrial.Center', 'Distance.to.Residential.Area',
       'Distance.to.Residential.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Water.Body', 'Distance.to.Water.Center',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'stv_nc_adb',
       'sep_bikeway_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'tertiary_bike_lane_binary', 'log_stv_adb', 'log_stv_c_adb',
       'log_stv_nc_adb', 'log_stl_raw', 'log_stv_stl',
       'Distance_to_Water_Body_mi', 'Intersection_Density_tm_net',
       'Commercial.Area_tm_net', 'Industrial.Area_tm_net',
       'Residential_Road_tm_net', 'Grass.Area_tm_net', 'School_tm_net',
       'Secondary_tm_net', 'Tertiary_tm_net', 'cycleway_lane_all_tm_net',
     

In [257]:
dadata0=da_data11[top_corr_features]
dadata00=dadata0.drop(['cycleway_lane_all_tm_net','stv_adb','stv_c_adb','log_stv_c_adb','log_stv_nc_adb','stl_raw','log_stv_stl','log_stv_adb','tertiary_no_bike_lane_binary','tertiary_bike_lane_binary'],axis=1)

In [258]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

dadata00
X_dataf = dadata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = dadata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 292 ms


,feature,VIF
8,BikeFac_binary,1.794927
6,Distance.to.forest,2.061009
4,Distance.to.Park,2.260036
14,pct_white_tm_net,2.303322
9,log_stl_raw,2.323821
2,Distance.to.Residential.Center,2.369220
13,maxspeed_tm_net,2.436899
5,Distance.to.Water.Center,2.592043
10,Industrial.Area_tm_net,2.813150
7,stv_nc_adb,2.818780


In [259]:
vif_data['feature']

1      Distance.to.Industrial.Center
2     Distance.to.Residential.Center
3                  Distance.to.Grass
4                   Distance.to.Park
5           Distance.to.Water.Center
6                 Distance.to.forest
7                         stv_nc_adb
8                     BikeFac_binary
9                        log_stl_raw
10            Industrial.Area_tm_net
11           Residential_Road_tm_net
12                   Tertiary_tm_net
13                   maxspeed_tm_net
14                  pct_white_tm_net
15                      slope_tm_net
Name: feature, dtype: object

# Boulder All buffer Model

In [263]:
bo_data=data[data['region']=='Boulder']
bo_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
63,64,P001BOU,B90004 US 36 North of Boulder,permanent,2019,Boulder,3.0,206.745402,102.547945,14.027397,...,NaN,0.000000,0.000000,4.544397,12.165644,53.529893,85.960580,3.691834,0.883893,64
64,65,P002BOU,Folsom at Boulder Creek,permanent,2019,Boulder,2.0,479.543452,29.739726,10.452055,...,NaN,0.000000,0.000000,21.179435,27.822404,48.958050,105.032549,0.903281,0.392384,65
65,66,P003BOU,Folsom at Pine,permanent,2019,Boulder,1.0,607.273809,52.397260,16.424658,...,NaN,0.183242,1.815874,10.706058,23.420727,49.016319,104.389696,1.048750,0.245676,66
66,67,P004BOU,Folsom at South St.,permanent,2019,Boulder,2.0,502.851786,29.904110,9.972603,...,NaN,0.085188,2.151470,6.359348,32.578822,46.306943,99.337266,0.929956,0.572558,67
67,68,P005BOU,Eco-Totem 13th at Walnut,permanent,2019,Boulder,2.0,601.530238,20.808219,9.095890,...,NaN,0.035249,4.509611,7.963230,15.517555,42.385664,121.454864,0.159742,0.398035,68


In [264]:
# Clean the data
an_data=bo_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['travel_pattern', 'Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Ori

In [265]:
bo1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
bo1_data=bo1_data.drop(['Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
bo1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,5.389072,10.029106,0.000000,0.000000,4.544397,12.165644,53.529893,85.960580,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,8.586719,12.012275,0.000000,0.000000,21.179435,27.822404,48.958050,105.032549,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,8.416046,12.393805,0.183242,1.815874,10.706058,23.420727,49.016319,104.389696,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,8.229778,11.660667,0.085188,2.151470,6.359348,32.578822,46.306943,99.337266,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,7.776115,10.858402,0.035249,4.509611,7.963230,15.517555,42.385664,121.454864,0.159742,0.398035


In [266]:
d1=bo1_data.loc[:,~bo1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,0,0,0,4.640035,2.709875,4.494468,5.389072,10.029106,3.691834,0.883893
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,0,0,0,3.425556,2.438169,3.010013,8.586719,12.012275,0.903281,0.392384
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,0,1,0,3.977759,2.857886,3.610177,8.416046,12.393805,1.048750,0.245676
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,0,1,0,3.430889,2.395402,3.041256,8.229778,11.660667,0.929956,0.572558
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,0,0,0,3.082287,2.312128,2.542572,7.776115,10.858402,0.159742,0.398035


In [267]:
net_data=bo1_data.loc[:,bo1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
63,0.458700,0.006019,1.312285,1.043868,346.831263,0.0,0.0,755.526231,0.000000,0.00000,...,37370.95158,74083.58393,107172.0786,0.000000,0.348678,3.102873,6.589070,11.188028,19.850355,29.026278
64,136.470572,17.000804,12.457487,750.423936,44.140496,0.0,0.0,88.614805,85978.230060,0.00000,...,76630.73908,148540.71380,222428.9833,0.546273,2.269906,7.029680,17.654888,31.858453,63.264569,88.564683
65,138.143676,12.400563,8.976569,454.909677,660.322790,0.0,0.0,2141.902304,0.000000,0.00000,...,49086.61710,145000.23470,298492.9677,0.810257,2.561413,10.083312,21.177660,34.247991,65.688353,104.476752
66,235.652820,44.874348,19.042568,1437.441934,320.325973,0.0,0.0,797.833163,155.075653,0.00000,...,67005.78732,142575.18400,290645.8543,0.764350,2.553340,9.104661,21.884627,35.423196,64.238280,103.224749
67,86.426472,8.437731,9.762901,243.609979,144.357518,0.0,0.0,1561.228770,258042.887600,11312.77801,...,44656.03747,109043.71570,177867.1421,0.411380,2.653661,7.503780,14.588088,23.645691,51.323599,76.235733


In [268]:
bo_data1=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
bo_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,37370.95158,74083.58393,107172.0786,0.000000,0.348678,3.102873,6.589070,11.188028,19.850355,29.026278
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,76630.73908,148540.71380,222428.9833,0.546273,2.269906,7.029680,17.654888,31.858453,63.264569,88.564683
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,49086.61710,145000.23470,298492.9677,0.810257,2.561413,10.083312,21.177660,34.247991,65.688353,104.476752
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,67005.78732,142575.18400,290645.8543,0.764350,2.553340,9.104661,21.884627,35.423196,64.238280,103.224749
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,44656.03747,109043.71570,177867.1421,0.411380,2.653661,7.503780,14.588088,23.645691,51.323599,76.235733


In [269]:
corrMatrix1 = bo_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.50]
top_corr_features

Index(['AADBT', 'stv_c_adb', 'stl_raw', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Retail.Area',
       'Distance.to.Retail.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park.Center',
       ...
       'sep_bikeway_om_net', 'sep_bikeway_ohm_net', 'sep_bikeway_tm_net',
       'BikeFac_otm_net', 'BikeFac_qm_net', 'BikeFac_hm_net',
       'BikeFac_tfm_net', 'BikeFac_om_net', 'BikeFac_ohm_net',
       'BikeFac_tm_net'],
      dtype='object', length=206)

In [270]:
bodata0=bo_data1[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass.Center'],axis=1)

In [271]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 12.1 s


,feature,VIF
6,sep_bikeway_otm_net,1.751882
4,pct_at_least_college_education_tm_net,2.195056
5,Median_HH_income_tm_net,2.214744
8,BikeFac_qm_net,2.413448
2,log_stv_c_adb,2.698123
3,pct_white_tm_net,2.797916
7,sep_bikeway_om_net,3.633366
1,stl_raw,4.317474


In [272]:
vif_data['feature']

1                                  stl_raw
2                            log_stv_c_adb
3                         pct_white_tm_net
4    pct_at_least_college_education_tm_net
5                  Median_HH_income_tm_net
6                      sep_bikeway_otm_net
7                       sep_bikeway_om_net
8                           BikeFac_qm_net
Name: feature, dtype: object

# Boulder 0.1 mile buffer

In [273]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,65.313228,165.473110,76.575350,0.690462,1.484353,47.49533,60.606044,0.021209,0.000000,0.000000
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3755.901854,11287.463570,6401.319674,152.964946,3.127135,47.49533,60.606044,0.021209,933.306149,0.546273
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,3589.245613,7441.048613,5017.901342,22.741271,0.664353,47.49533,60.606044,0.021209,73.815603,0.810257
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,5522.091477,11922.936350,8777.009693,598.460515,0.718577,47.49533,60.606044,0.021209,222.607567,0.764350
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,1722.670565,3800.315671,2634.467915,2617.428072,1.200773,47.49533,60.606044,0.021209,2172.086436,0.411380


In [274]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [275]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['tertiary_no_bike_lane_binary','secondary_bike_lane_binary','tertiary_bike_lane_binary','Distance.to.Grass', 'stv_nc_adb'],axis=1)

In [276]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.03 s


,feature,VIF
1,Point.Speed,1.448910
4,log_stv_nc_adb,1.568720
14,Median.Age_otm_net,1.709151
9,Forest.Area_otm_net,1.930932
7,Retail.Area_otm_net,2.439132
11,Primary_otm_net,2.525724
3,sep_onstreet_binary,2.675185
5,Bike.Commuter_otm_net,2.730868
13,cycleway_track_all_otm_net,2.761562
8,Park.Area_otm_net,2.911985


In [277]:
vif_data['feature']

1                    Point.Speed
2                   Point.Bridge
3            sep_onstreet_binary
4                 log_stv_nc_adb
5          Bike.Commuter_otm_net
6       Residential_Road_otm_net
7            Retail.Area_otm_net
8              Park.Area_otm_net
9            Forest.Area_otm_net
10      OSM_node_Density_otm_net
11               Primary_otm_net
12             Secondary_otm_net
13    cycleway_track_all_otm_net
14            Median.Age_otm_net
15               BikeFac_otm_net
Name: feature, dtype: object

# Boulder 0.25 mile buffer

In [278]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,852.743592,1756.687653,1242.584290,44.630167,1.492417,47.49533,60.606044,0.021209,229.458818,0.348678
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3708.236001,10583.248400,5853.457142,1083.401273,1.903073,47.49533,60.606044,0.021209,6198.752971,2.269906
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,3549.116220,7505.451108,5024.007773,798.780601,1.050931,47.49533,60.606044,0.021209,1726.523305,2.561413
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,5506.300012,11729.298210,8310.067279,2353.466306,0.890593,47.49533,60.606044,0.021209,1082.637253,2.553340
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,1802.013883,4034.689240,2713.288496,11304.284700,1.401676,47.49533,60.606044,0.021209,9992.822340,2.653661


In [279]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [294]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['tertiary_no_bike_lane_binary','Distance.to.Grass','cycleway_track_all_qm_net','OSM_node_Density_qm_net','cycleway_lane_all_qm_net','tertiary_bike_lane_binary'],axis=1)

In [295]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.15 s


,feature,VIF
3,log_stv_nc_adb,1.581818
15,slope_qm_net,1.635001
8,School_qm_net,1.721112
1,sep_onstreet_binary,1.914805
16,sep_bikeway_qm_net,2.246269
4,Intersection_Density_qm_net,2.325053
2,BikeFac_binary,2.546219
7,Park.Area_qm_net,2.957835
11,bridge_qm_net,2.981709
14,Number.of.jobs_qm_net,3.149317


In [296]:
vif_data['feature']

1             sep_onstreet_binary
2                  BikeFac_binary
3                  log_stv_nc_adb
4     Intersection_Density_qm_net
5         Residential_Road_qm_net
6              Retail.Area_qm_net
7                Park.Area_qm_net
8                   School_qm_net
9                 Tertiary_qm_net
10                 Footway_qm_net
11                  bridge_qm_net
12    pct_African_American_qm_net
13        Median_HH_income_qm_net
14          Number.of.jobs_qm_net
15                   slope_qm_net
16             sep_bikeway_qm_net
Name: feature, dtype: object

# Boulder 0.5 mile buffer

In [297]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,823.236302,1717.360975,1193.074555,438.077841,1.926347,47.49533,60.606044,0.021209,10941.79547,3.102873
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3833.066719,11257.615720,6149.962285,3768.223916,1.629683,47.49533,60.606044,0.021209,21074.28780,7.029680
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,3841.699431,7985.132626,5561.222703,3751.954466,1.202834,47.49533,60.606044,0.021209,12349.67434,10.083312
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,4701.930358,10012.802530,6795.506117,6597.441082,1.120174,47.49533,60.606044,0.021209,10226.59720,9.104661
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,3476.283873,7660.310448,4697.049054,17661.901150,1.671476,47.49533,60.606044,0.021209,17048.62594,7.503780


In [298]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [299]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop([ 'Distance.to.Grass.Center'],axis=1)

In [300]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.39 s


,feature,VIF
13,pct_African_American_hm_net,1.383433
12,bridge_hm_net,1.509586
2,log_stv_nc_adb,1.663353
9,University_hm_net,1.760848
6,Park.Area_hm_net,1.770937
14,Median_HH_income_hm_net,2.181663
15,slope_hm_net,2.193040
1,BikeFac_binary,2.514873
8,School_hm_net,2.864285
5,Retail.Area_hm_net,2.963286


In [301]:
vif_data['feature']

1                  BikeFac_binary
2                  log_stv_nc_adb
3                     log_stl_raw
4     Intersection_Density_hm_net
5              Retail.Area_hm_net
6                Park.Area_hm_net
7              Forest.Area_hm_net
8                   School_hm_net
9               University_hm_net
10        OSM_node_Density_hm_net
11      cycleway_track_all_hm_net
12                  bridge_hm_net
13    pct_African_American_hm_net
14        Median_HH_income_hm_net
15                   slope_hm_net
Name: feature, dtype: object

# Boulder 0.75 mile Boulder Model

In [302]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,1244.191123,2616.502510,1782.258479,1249.306756,2.085024,47.49533,60.606044,0.021209,23338.13774,6.589070
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3800.600717,11041.805830,6060.438000,7732.118639,1.956562,47.49533,60.606044,0.021209,44503.69156,17.654888
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,3625.844421,7620.526663,5171.256334,10059.598230,1.296300,47.49533,60.606044,0.021209,26461.73420,21.177660
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,4044.327686,8973.202212,5866.026347,12623.998280,1.274729,47.49533,60.606044,0.021209,38598.34186,21.884627
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,4228.237580,10016.439820,6044.147086,20421.481000,2.076005,47.49533,60.606044,0.021209,26299.97634,14.588088


In [303]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [305]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass'],axis=1)

In [306]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.45 s


,feature,VIF
10,pct_African_American_tfm_net,1.303898
9,bridge_tfm_net,1.389543
12,slope_tfm_net,1.415904
8,cycleway_track_all_tfm_net,1.656274
1,log_stv_nc_adb,1.721544
3,Intersection_Density_tfm_net,2.017527
6,college_tfm_net,2.213592
5,Park.Area_tfm_net,2.317275
4,Grass.Area_tfm_net,2.392993
7,Tertiary_tfm_net,2.582400


In [307]:
vif_data['feature']

1                   log_stv_nc_adb
2                      log_stl_raw
3     Intersection_Density_tfm_net
4               Grass.Area_tfm_net
5                Park.Area_tfm_net
6                  college_tfm_net
7                 Tertiary_tfm_net
8       cycleway_track_all_tfm_net
9                   bridge_tfm_net
10    pct_African_American_tfm_net
11        Median_HH_income_tfm_net
12                   slope_tfm_net
Name: feature, dtype: object

# Boulder 1 mile Buffer

In [308]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,1212.706425,2581.540372,1665.037142,1541.356339,2.183730,47.49533,60.606044,0.021209,37370.95158,11.188028
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3768.193427,10368.716590,5631.621614,15401.772950,2.005360,47.49533,60.606044,0.021209,76630.73908,31.858453
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,3332.623592,7235.615293,4716.696901,21190.047460,1.358542,47.49533,60.606044,0.021209,49086.61710,34.247991
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,3304.360244,7867.979185,4849.715738,23893.954960,1.532682,47.49533,60.606044,0.021209,67005.78732,35.423196
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,3800.115635,9777.857345,5753.658631,24035.784020,2.307083,47.49533,60.606044,0.021209,44656.03747,23.645691


In [309]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [310]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['Point.Bridge','sep_bikeway_binary','secondary_bike_lane_binary','Distance.to.Grass', 'Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','sep_onstreet_binary'],axis=1)

In [311]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.19 s


,feature,VIF
10,pct_African_American_om_net,1.368723
1,stv_nc_adb,1.372269
8,bridge_om_net,1.484127
13,slope_om_net,1.610235
7,cycleway_track_all_om_net,1.694984
3,Commercial.Area_om_net,1.901033
11,pct_at_least_college_education_om_net,2.355879
2,Intersection_Density_om_net,2.566543
6,Tertiary_om_net,2.617652
4,Grass.Area_om_net,2.999789


In [312]:
vif_data['feature']

1                                stv_nc_adb
2               Intersection_Density_om_net
3                    Commercial.Area_om_net
4                         Grass.Area_om_net
5                          Park.Area_om_net
6                           Tertiary_om_net
7                 cycleway_track_all_om_net
8                             bridge_om_net
9                          pct_white_om_net
10              pct_African_American_om_net
11    pct_at_least_college_education_om_net
12                  Median_HH_income_om_net
13                             slope_om_net
Name: feature, dtype: object

# Boulder 1.5 miles Buffer

In [314]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,1170.266266,2629.677455,1496.971460,1928.599535,2.267873,47.49533,60.606044,0.021209,74083.58393,19.850355
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,3264.641264,8771.198647,4807.770697,46124.634530,1.998339,47.49533,60.606044,0.021209,148540.71380,63.264569
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,2791.204782,6526.514931,3876.785694,48645.331000,1.721922,47.49533,60.606044,0.021209,145000.23470,65.688353
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,3098.584293,7698.802326,4370.983131,51783.888800,1.937631,47.49533,60.606044,0.021209,142575.18400,64.238280
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,2914.001402,7773.028985,4387.367441,33408.519730,2.367225,47.49533,60.606044,0.021209,109043.71570,51.323599


In [315]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [317]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','stv_nc_adb'],axis=1)

In [318]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.36 s


,feature,VIF
6,maxspeed_ohm_net,1.589050
1,log_stv_nc_adb,1.614898
10,slope_ohm_net,2.259569
2,Commercial.Area_ohm_net,2.655766
5,cycleway_track_all_ohm_net,2.672400
4,OSM_node_Density_ohm_net,3.046100
8,pct_at_least_college_education_ohm_net,3.304555
11,sep_bikeway_ohm_net,3.364930
3,college_ohm_net,3.439247
7,pct_white_ohm_net,4.560490


In [319]:
vif_data['feature']

1                             log_stv_nc_adb
2                    Commercial.Area_ohm_net
3                            college_ohm_net
4                   OSM_node_Density_ohm_net
5                 cycleway_track_all_ohm_net
6                           maxspeed_ohm_net
7                          pct_white_ohm_net
8     pct_at_least_college_education_ohm_net
9                   Median_HH_income_ohm_net
10                             slope_ohm_net
11                       sep_bikeway_ohm_net
Name: feature, dtype: object

# Boulder 2 miles Buffer

In [320]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
bo_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
bo_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
63,206.745402,102.547945,14.027397,218,1,0,0,0,0,0,...,1155.839615,2740.320228,1490.450605,2554.479861,2.423815,47.49533,60.606044,0.021209,107172.0786,29.026278
64,479.543452,29.739726,10.452055,5359,0,0,1,0,0,0,...,2785.648577,7224.785255,4063.306279,58132.980240,2.094391,47.49533,60.606044,0.021209,222428.9833,88.564683
65,607.273809,52.397260,16.424658,4518,0,1,0,0,0,0,...,2694.717275,6607.534501,3766.916857,57798.418150,1.989286,47.49533,60.606044,0.021209,298492.9677,104.476752
66,502.851786,29.904110,9.972603,3750,0,1,0,0,0,0,...,2782.567924,6990.853506,3953.941939,58957.757850,2.011454,47.49533,60.606044,0.021209,290645.8543,103.224749
67,601.530238,20.808219,9.095890,2382,0,0,0,1,0,0,...,2647.710091,6787.689421,3795.360291,47407.504790,2.385138,47.49533,60.606044,0.021209,177867.1421,76.235733


In [321]:
corrMatrix1 = bo_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'stv_c_adb', 'stl_raw', 'tertiary_binary',
       'cycleway_lane_binary', 'cycleway_track_all_binary',
       'Distance.to.Commercial.Area', 'Distance.to.Commercial.Area.Center',
       'Distance.to.Industrial.Area', 'Distance.to.Industrial.Center',
       'Distance.to.Residential.Area', 'Distance.to.Residential.Center',
       'Distance.to.Retail.Area', 'Distance.to.Retail.Center',
       'Distance.to.Grass', 'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.forest',
       'Distance.to.Forest.Center', 'Distance.to.CBD', 'min_dist_to_school',
       'min_dist_to_college', 'min_dist_to_university', 'Point.Speed',
       'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'tertiary_no_bike_lane_binary',
       'secondary_bike_lane_binary', 'tertiary_bike_lane_binary',
       'log_stv_adb', 'log_stv_c_adb', 'log_stv_nc_adb', 'log_stl_raw',
       'log_stv_stl', 'Distanc

In [324]:
bodata0=bo_data11[top_corr_features]
bodata00=bodata0.drop(['secondary_bike_lane_binary', 'Distance.to.Grass.Center','cycleway_track_all_binary','tertiary_bike_lane_binary','tertiary_no_bike_lane_binary','stv_nc_adb'],axis=1)

In [325]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

bodata00
X_dataf = bodata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = bodata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 1.34 s


,feature,VIF
6,pct_African_American_tm_net,1.944784
1,log_stv_nc_adb,2.119461
4,maxspeed_tm_net,2.256369
9,slope_tm_net,2.935166
7,pct_at_least_college_education_tm_net,3.172553
8,Median_HH_income_tm_net,3.214480
10,sep_bikeway_tm_net,3.220842
3,OSM_node_Density_tm_net,3.375097
2,Industrial.Area_tm_net,3.441083
5,pct_white_tm_net,4.055547


In [326]:
vif_data['feature']

1                            log_stv_nc_adb
2                    Industrial.Area_tm_net
3                   OSM_node_Density_tm_net
4                           maxspeed_tm_net
5                          pct_white_tm_net
6               pct_African_American_tm_net
7     pct_at_least_college_education_tm_net
8                   Median_HH_income_tm_net
9                              slope_tm_net
10                       sep_bikeway_tm_net
Name: feature, dtype: object

# Charlotte All Buffers

In [10]:
ch_data=data[data['region']=='Charlotte']
ch_data.head()

,Unnamed: 0,site_id,site_name,type,year,region,travel_pattern,AADBT,stv_adb,stv_c_adb,...,Bike.Share.best_tm,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi,rowIndex
102,103,P001CHA,Blue Line Trail,permanent,2019,Charlotte,3.0,393.654456,18.547945,5.958904,...,NaN,0.0000,0.000000,0.041610,33.852053,85.171864,198.519985,1.556811,0.766116,103
103,104,P002CHA,Pecan Ave,permanent,2019,Charlotte,3.0,160.742460,16.082192,4.602740,...,NaN,0.0000,0.000000,2.249798,39.927741,81.717367,197.069588,1.371700,0.347870,104
104,105,P003CHA,Selwyn Avenue,permanent,2019,Charlotte,3.0,39.364881,15.465753,2.945205,...,NaN,0.0000,0.000000,4.910771,29.877785,80.588010,136.432429,3.008143,0.482522,105
105,106,P004CHA,South Tryon Street,permanent,2019,Charlotte,1.0,149.292520,11.630137,4.780822,...,NaN,0.0000,0.617506,1.755423,8.325280,69.409236,156.062736,0.690422,0.370499,106
106,107,P005CHA,Stonewall Street,permanent,2019,Charlotte,2.0,33.307341,2.438356,1.136986,...,NaN,0.5032,17.150246,41.189231,77.882806,112.212650,180.740409,0.509815,0.247150,107


In [11]:
# Clean the data
an_data=ch_data._get_numeric_data()
nan_values = an_data. isna()
nan_columns = nan_values. any()
columns_with_nan = an_data. columns[nan_columns]. tolist()
print(columns_with_nan)

['Bike.Share.Origin_otm', 'Bike.Share.Destination_otm', 'Bike.Share.Crossing_otm', 'Bike.Share.route_otm', 'Bike.Share.Origin_qm', 'Bike.Share.Destination_qm', 'Bike.Share.Crossing_qm', 'Bike.Share.route_qm', 'Bike.Share.Origin_hm', 'Bike.Share.Destination_hm', 'Bike.Share.Crossing_hm', 'Bike.Share.route_hm', 'Bike.Share.Origin_tfm', 'Bike.Share.Destination_tfm', 'Bike.Share.Crossing_tfm', 'Bike.Share.route_tfm', 'Bike.Share.Origin_om', 'Bike.Share.Destination_om', 'Bike.Share.Crossing_om', 'Bike.Share.route_om', 'Bike.Share.Origin_ohm', 'Bike.Share.Destination_ohm', 'Bike.Share.Crossing_ohm', 'Bike.Share.route_ohm', 'Bike.Share.Origin_tm', 'Bike.Share.Destination_tm', 'Bike.Share.Crossing_tm', 'Bike.Share.route_tm', 'Bike.Share.Origin_otm_net', 'Bike.Share.Destination_otm_net', 'Bike.Share.Crossing_otm_net', 'Bike.Share.route_otm_net', 'Bike.Share.Origin_qm_net', 'Bike.Share.Destination_qm_net', 'Bike.Share.Crossing_qm_net', 'Bike.Share.route_qm_net', 'Bike.Share.Origin_hm_net', 'Bike

In [12]:
ch1_data=an_data.drop(an_data. columns[nan_columns]. tolist(),axis=1)
ch1_data=ch1_data.drop(['travel_pattern','Unnamed: 0','year','latitude','longitude','rowIndex'],axis=1)
ch1_data.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,log_stl_raw,log_stv_stl,Park_acres_otm,Park_acres_qm,Park_acres_hm,Park_acres_tfm,Park_acres_om,Park_acres_ohm,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,7.120444,10.093315,0.0000,0.000000,0.041610,33.852053,85.171864,198.519985,1.556811,0.766116
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,6.608001,9.446037,0.0000,0.000000,2.249798,39.927741,81.717367,197.069588,1.371700,0.347870
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,5.796058,8.597340,0.0000,0.000000,4.910771,29.877785,80.588010,136.432429,3.008143,0.482522
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,7.625595,10.161681,0.0000,0.617506,1.755423,8.325280,69.409236,156.062736,0.690422,0.370499
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,7.275172,8.510166,0.5032,17.150246,41.189231,77.882806,112.212650,180.740409,0.509815,0.247150


In [13]:
d1=ch1_data.loc[:,~ch1_data.columns.str.contains('_otm', case=False)]
d2=d1.loc[:,~d1.columns.str.contains('_qm', case=False)]
d3=d2.loc[:,~d2.columns.str.contains('_hm', case=False)]
d4=d3.loc[:,~d3.columns.str.contains('_tfm', case=False)]
d5=d4.loc[:,~d4.columns.str.contains('_om', case=False)]
d6=d5.loc[:,~d5.columns.str.contains('_ohm', case=False)]
d7=d6.loc[:,~d6.columns.str.contains('_tm', case=False)]
d7.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,primary_bike_lane_binary,secondary_bike_lane_binary,tertiary_bike_lane_binary,log_stv_adb,log_stv_c_adb,log_stv_nc_adb,log_stl_raw,log_stv_stl,Distance_to_CBD_mi,Distance_to_Water_Body_mi
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,0,0,0,2.972870,1.940022,2.609264,7.120444,10.093315,1.556811,0.766116
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,0,0,1,2.838037,1.723256,2.524083,6.608001,9.446037,1.371700,0.347870
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,0,0,0,2.801283,1.372501,2.604211,5.796058,8.597340,3.008143,0.482522
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,0,0,0,2.536086,1.754546,2.060426,7.625595,10.161681,0.690422,0.370499
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,0,0,0,1.234993,0.759397,0.833505,7.275172,8.510166,0.509815,0.247150


In [14]:
net_data=ch1_data.loc[:,ch1_data.columns.str.contains('_net', case=False)]
net_data.head()

,Total.Commuter_otm_net,Bike.Commuter_otm_net,Percentage.of.Bike.Commuter_otm_net,bike.commuter.density_otm_net,Intersection_Density_otm_net,Commercial.Area_otm_net,Industrial.Area_otm_net,Residential_Road_otm_net,Retail.Area_otm_net,Grass.Area_otm_net,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
102,75.840312,0.000000,0.000000,0.000000,149.901579,63225.01045,0.0,407.061649,37311.66903,5920.668973,...,5089.053649,5089.053649,5089.053649,0.098960,0.398951,0.963836,3.128469,0.963836,0.963836,0.963836
103,59.728070,0.000000,0.000000,0.000000,225.351460,0.00000,0.0,477.229909,498130.27110,0.000000,...,2583.938599,2583.938599,2583.938599,0.000000,0.093009,0.489382,4.462587,0.489382,0.489382,0.489382
104,42.050837,0.334605,0.795716,19.553626,175.313638,0.00000,0.0,1270.182815,0.00000,0.000000,...,19805.975640,19805.975640,19805.975640,0.315713,1.715807,5.135609,12.351658,5.135609,5.135609,5.135609
105,104.815435,0.000000,0.000000,0.000000,201.181643,0.00000,0.0,343.633518,0.00000,0.000000,...,6110.769132,6110.769132,6110.769132,0.011574,0.169129,1.816037,3.935852,1.816037,1.816037,1.816037
106,19.830763,0.000000,0.000000,0.000000,349.979314,0.00000,0.0,110.479404,0.00000,0.000000,...,14287.398580,14287.398580,14287.398580,0.550977,1.529594,3.559167,9.156019,3.559167,3.559167,3.559167


In [15]:
ch_data1=pd.concat([d7, net_data.reindex(d7.index)], axis=1)
ch_data1.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,sep_bikeway_om_net,sep_bikeway_ohm_net,sep_bikeway_tm_net,BikeFac_otm_net,BikeFac_qm_net,BikeFac_hm_net,BikeFac_tfm_net,BikeFac_om_net,BikeFac_ohm_net,BikeFac_tm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,5089.053649,5089.053649,5089.053649,0.098960,0.398951,0.963836,3.128469,0.963836,0.963836,0.963836
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2583.938599,2583.938599,2583.938599,0.000000,0.093009,0.489382,4.462587,0.489382,0.489382,0.489382
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,19805.975640,19805.975640,19805.975640,0.315713,1.715807,5.135609,12.351658,5.135609,5.135609,5.135609
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,6110.769132,6110.769132,6110.769132,0.011574,0.169129,1.816037,3.935852,1.816037,1.816037,1.816037
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,14287.398580,14287.398580,14287.398580,0.550977,1.529594,3.559167,9.156019,3.559167,3.559167,3.559167


In [16]:
corrMatrix1 = ch_data1.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.40]
top_corr_features

Index(['AADBT', 'secondary_binary', 'cycleway_binary', 'Distance.to.Park',
       'Distance.to.forest', 'Distance.to.Forest.Center',
       'min_dist_to_university', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary',
       ...
       'population_density_tm_net', 'employment_density_tm_net',
       'slope_tm_net', 'BikeFac_otm_net', 'BikeFac_qm_net', 'BikeFac_hm_net',
       'BikeFac_tfm_net', 'BikeFac_om_net', 'BikeFac_ohm_net',
       'BikeFac_tm_net'],
      dtype='object', length=154)

In [17]:
chdata0=ch_data1[top_corr_features]
chdata00=chdata0.drop(['secondary_no_bike_lane_binary','arterial_no_bike_lane_binary'],axis=1)

In [18]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 4.09 s


,feature,VIF
6,slope_tm_net,1.377833
3,maxspeed_tm_net,1.748658
5,HH_density_tm_net,2.145939
7,BikeFac_tfm_net,2.196569
1,log_stv_nc_adb,2.494907
2,cycleway_lane_all_tm_net,2.978905
4,Student.Access_tm_net,4.456509


In [19]:
vif_data['feature']

1              log_stv_nc_adb
2    cycleway_lane_all_tm_net
3             maxspeed_tm_net
4       Student.Access_tm_net
5           HH_density_tm_net
6                slope_tm_net
7             BikeFac_tfm_net
Name: feature, dtype: object

# Charlotte 0.1 mile buffer

In [20]:
d1=net_data.loc[:,net_data.columns.str.contains('_otm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_otm_net,population_density_otm_net,employment_density_otm_net,Number.of.jobs_otm_net,slope_otm_net,Temp_avg_otm_net,Hum_avg_otm_net,Precipitation_otm_net,sep_bikeway_otm_net,BikeFac_otm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2768.016427,5479.008728,3879.054555,570.079300,1.185959,63.593151,65.583288,0.148247,522.509188,0.098960
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,1834.777311,3535.999049,2813.487726,210.639017,1.109358,63.593151,65.583288,0.148247,0.000000,0.000000
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,2056.827324,5072.418822,2479.085659,14.376865,0.000000,63.593151,65.583288,0.148247,0.000000,0.315713
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,4263.609219,6204.911305,5271.735365,445.766027,0.000000,63.593151,65.583288,0.148247,61.112751,0.011574
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1603.115448,2257.449475,1455.149318,124.324401,2.909993,63.593151,65.583288,0.148247,394.638257,0.550977


In [25]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.15]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'Point.Speed', 'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'arterial_binary',
       'arterial_no_bike_lane_binary', 'secondary_no_bike_lane_binary',
       'log_stv_adb', 'log_stv_nc_adb', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_otm_net',
       'Bike.Commuter_otm_net', 'Percentage.of.Bike.Commuter_otm_net',
       'bike.commuter.density_otm_net', 'Intersection_Density_otm_net',
       'Residential_Road_otm_net', '

In [26]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop([ 'Distance.to.Grass.Center','arterial_no_bike_lane_binary', 'secondary_no_bike_lane_binary'],axis=1)

In [27]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 670 ms


,feature,VIF
2,pct_female_otm_net,1.128696
5,Median.Age_otm_net,1.230554
7,slope_otm_net,1.527834
4,pct_at_least_college_education_otm_net,2.502366
6,Number.of.jobs_otm_net,3.063188
8,BikeFac_otm_net,3.251082
3,Student.Access_otm_net,3.542528
1,log_stv_stl,4.308623


In [28]:
vif_data['feature']

1                               log_stv_stl
2                        pct_female_otm_net
3                    Student.Access_otm_net
4    pct_at_least_college_education_otm_net
5                        Median.Age_otm_net
6                    Number.of.jobs_otm_net
7                             slope_otm_net
8                           BikeFac_otm_net
Name: feature, dtype: object

# Charlotte 0.25 mile buffer

In [29]:
d1=net_data.loc[:,net_data.columns.str.contains('_qm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_qm_net,population_density_qm_net,employment_density_qm_net,Number.of.jobs_qm_net,slope_qm_net,Temp_avg_qm_net,Hum_avg_qm_net,Precipitation_qm_net,sep_bikeway_qm_net,BikeFac_qm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2533.867896,5152.873483,3591.738396,2513.045164,1.189158,63.593151,65.583288,0.148247,2106.460676,0.398951
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2131.690942,4218.208037,3283.847810,1174.221908,1.364475,63.593151,65.583288,0.148247,491.088064,0.093009
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1834.707453,4525.662356,2141.067716,161.418677,0.000000,63.593151,65.583288,0.148247,3890.860170,1.715807
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,3908.588053,5785.663592,4905.907640,2882.329369,0.000000,63.593151,65.583288,0.148247,893.000817,0.169129
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1634.121194,2384.894393,1519.433562,1754.364840,2.365606,63.593151,65.583288,0.148247,3571.251194,1.529594


In [30]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_qm_net', 'Bike.Commuter_qm_net',
       'Percentage.of.Bike.Commuter_qm_net', 'bike.commuter.density_qm_net',
       'Intersection_Density_qm_net', 'Residential_Road_qm_net',
       'Grass.Area_qm_net', 'Park.Area_qm_net', 'Wat

In [31]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop([ 'Distance.to.Grass.Center','cycleway_track_all_binary','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary'],axis=1)

In [32]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 540 ms


,feature,VIF
3,pct_female_qm_net,1.159899
5,slope_qm_net,1.194334
1,log_stv_stl,1.196978
4,population_density_qm_net,1.725825
2,pct_African_American_qm_net,2.057718
6,BikeFac_qm_net,2.369107


In [33]:
vif_data['feature']

1                    log_stv_stl
2    pct_African_American_qm_net
3              pct_female_qm_net
4      population_density_qm_net
5                   slope_qm_net
6                 BikeFac_qm_net
Name: feature, dtype: object

# Charlotte 0.50 mile buffer

In [34]:
d1=net_data.loc[:,net_data.columns.str.contains('_hm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_hm_net,population_density_hm_net,employment_density_hm_net,Number.of.jobs_hm_net,slope_hm_net,Temp_avg_hm_net,Hum_avg_hm_net,Precipitation_hm_net,sep_bikeway_hm_net,BikeFac_hm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2832.647051,5514.436317,3921.882475,5958.049647,1.549387,63.593151,65.583288,0.148247,5089.053649,0.963836
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2401.961584,4781.998045,3558.593176,2386.008587,2.084362,63.593151,65.583288,0.148247,2583.938599,0.489382
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1852.537683,4701.628327,2237.932328,835.356120,0.000000,63.593151,65.583288,0.148247,19805.975640,5.135609
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,4091.954640,6386.140231,5398.276665,26461.628710,0.000000,63.593151,65.583288,0.148247,6110.769132,1.816037
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1748.502401,2752.939974,1733.910449,9564.959850,2.263852,63.593151,65.583288,0.148247,14287.398580,3.559167


In [35]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_hm_net', 'Bike.Commuter_hm_net',
       'Percentage.of.Bike.Commuter_hm_net', 'bike.commuter.density_hm_net',
       'Intersection_Density_hm_net', 'Residential_Road_hm_net',
       'Grass.Area_hm_net', 'Park.Area_hm_net', 'Wat

In [36]:
chdata0=ch_data1[top_corr_features]
chdata00=chdata0.drop(['Distance.to.Grass', 'Distance.to.Grass.Center','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary','OSM_node_Density_hm_net'],axis=1)

In [37]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 586 ms


,feature,VIF
6,slope_hm_net,1.177898
2,pct_African_American_hm_net,1.494980
7,sep_bikeway_hm_net,2.066031
3,pct_female_hm_net,2.116726
1,log_stv_stl,3.307308
5,Number.of.jobs_hm_net,4.055736
4,Student.Access_hm_net,4.511726


In [38]:
vif_data['feature']

1                    log_stv_stl
2    pct_African_American_hm_net
3              pct_female_hm_net
4          Student.Access_hm_net
5          Number.of.jobs_hm_net
6                   slope_hm_net
7             sep_bikeway_hm_net
Name: feature, dtype: object

# Charlotte 0.75 mile buffer

In [359]:
d1=net_data.loc[:,net_data.columns.str.contains('_tfm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tfm_net,population_density_tfm_net,employment_density_tfm_net,Number.of.jobs_tfm_net,slope_tfm_net,Temp_avg_tfm_net,Hum_avg_tfm_net,Precipitation_tfm_net,sep_bikeway_tfm_net,BikeFac_tfm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,3068.733545,5742.104249,4182.670960,9205.527589,2.000994,63.593151,65.583288,0.148247,10139.92060,3.128469
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2433.464934,4860.795632,3518.355867,4335.227267,2.227619,63.593151,65.583288,0.148247,21078.19530,4.462587
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1807.480423,4645.721546,2243.874641,1320.287803,0.000000,63.593151,65.583288,0.148247,57906.71047,12.351658
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,3664.976065,6074.181206,4938.368981,62897.739700,0.000000,63.593151,65.583288,0.148247,12191.67515,3.935852
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1850.983611,3190.011736,2056.064785,29861.668300,2.439724,63.593151,65.583288,0.148247,42770.44932,9.156019


In [364]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.15]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_college', 'min_dist_to_university',
       'Point.Speed', 'Point.Bridge', 'stv_nc_adb', 'sep_bikeway_binary',
       'sep_onstreet_binary', 'BikeFac_binary', 'arterial_binary',
       'arterial_no_bike_lane_binary', 'secondary_no_bike_lane_binary',
       'log_stv_adb', 'log_stv_nc_adb', 'log_stv_stl', 'Distance_to_CBD_mi',
       'Distance_to_Water_Body_mi', 'Total.Commuter_tfm_net',
       'Bike.Commuter_tfm_net', 'Percentage.of.Bike.Commuter_tfm_net',
       'bike.commuter.density_tfm_net', 'Intersection_Density_tfm_net',
       'Residential_Road_tfm_net', '

In [365]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop([ 'Distance.to.Grass.Center','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary','cycleway_track_all_binary'],axis=1)

In [366]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 690 ms


,feature,VIF
6,slope_tfm_net,1.135654
1,log_stv_stl,1.288952
7,sep_bikeway_tfm_net,1.483277
5,Median.Age_tfm_net,1.830992
3,pct_African_American_tfm_net,1.849718
4,pct_female_tfm_net,2.029869
2,maxspeed_tfm_net,2.052001


In [367]:
vif_data['feature']

1                     log_stv_stl
2                maxspeed_tfm_net
3    pct_African_American_tfm_net
4              pct_female_tfm_net
5              Median.Age_tfm_net
6                   slope_tfm_net
7             sep_bikeway_tfm_net
Name: feature, dtype: object

# Charlotte 1.0 mile buffer

In [368]:
d1=net_data.loc[:,net_data.columns.str.contains('_om_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_om_net,population_density_om_net,employment_density_om_net,Number.of.jobs_om_net,slope_om_net,Temp_avg_om_net,Hum_avg_om_net,Precipitation_om_net,sep_bikeway_om_net,BikeFac_om_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2832.647051,5514.436317,3921.882475,5958.049647,1.549387,63.593151,65.583288,0.148247,5089.053649,0.963836
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2401.961584,4781.998045,3558.593176,2386.008587,2.084362,63.593151,65.583288,0.148247,2583.938599,0.489382
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1852.537683,4701.628327,2237.932328,835.356120,0.000000,63.593151,65.583288,0.148247,19805.975640,5.135609
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,4091.954640,6386.140231,5398.276665,26461.628710,0.000000,63.593151,65.583288,0.148247,6110.769132,1.816037
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1748.502401,2752.939974,1733.910449,9564.959850,2.263852,63.593151,65.583288,0.148247,14287.398580,3.559167


In [369]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_om_net', 'Bike.Commuter_om_net',
       'Percentage.of.Bike.Commuter_om_net', 'bike.commuter.density_om_net',
       'Intersection_Density_om_net', 'Residential_Road_om_net',
       'Grass.Area_om_net', 'Park.Area_om_net', 'Wat

In [371]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop(['cycleway_lane_all_om_net', 'cycleway_track_all_om_net','cycleway_lane_all_om_net', 'cycleway_track_all_om_net','cycleway_track_all_binary','Distance.to.Grass','arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary'],axis=1)

In [372]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 603 ms


,feature,VIF
6,slope_om_net,1.272552
1,log_stv_stl,1.384727
3,pct_African_American_om_net,1.696216
2,maxspeed_om_net,1.865424
7,sep_bikeway_om_net,2.194334
4,pct_female_om_net,2.267989
5,population_density_om_net,2.352636


In [373]:
vif_data['feature']

1                    log_stv_stl
2                maxspeed_om_net
3    pct_African_American_om_net
4              pct_female_om_net
5      population_density_om_net
6                   slope_om_net
7             sep_bikeway_om_net
Name: feature, dtype: object

# Charlotte1.5 mile buffer

In [374]:
d1=net_data.loc[:,net_data.columns.str.contains('_ohm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_ohm_net,population_density_ohm_net,employment_density_ohm_net,Number.of.jobs_ohm_net,slope_ohm_net,Temp_avg_ohm_net,Hum_avg_ohm_net,Precipitation_ohm_net,sep_bikeway_ohm_net,BikeFac_ohm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2832.647051,5514.436317,3921.882475,5958.049647,1.549387,63.593151,65.583288,0.148247,5089.053649,0.963836
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2401.961584,4781.998045,3558.593176,2386.008587,2.084362,63.593151,65.583288,0.148247,2583.938599,0.489382
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1852.537683,4701.628327,2237.932328,835.356120,0.000000,63.593151,65.583288,0.148247,19805.975640,5.135609
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,4091.954640,6386.140231,5398.276665,26461.628710,0.000000,63.593151,65.583288,0.148247,6110.769132,1.816037
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1748.502401,2752.939974,1733.910449,9564.959850,2.263852,63.593151,65.583288,0.148247,14287.398580,3.559167


In [375]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_ohm_net', 'Bike.Commuter_ohm_net',
       'Percentage.of.Bike.Commuter_ohm_net', 'bike.commuter.density_ohm_net',
       'Intersection_Density_ohm_net', 'Residential_Road_ohm_net',
       'Grass.Area_ohm_net', 'Park.Area_ohm_ne

In [376]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop(['Distance.to.Grass.Center','cycleway_track_all_binary','secondary_no_bike_lane_binary','arterial_no_bike_lane_binary','arterial_no_bike_lane_binary'],axis=1)

In [377]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 652 ms


,feature,VIF
6,slope_ohm_net,1.272552
1,log_stv_stl,1.384727
3,pct_African_American_ohm_net,1.696216
2,maxspeed_ohm_net,1.865424
7,sep_bikeway_ohm_net,2.194334
4,pct_female_ohm_net,2.267989
5,population_density_ohm_net,2.352636


In [378]:
vif_data['feature']

1                     log_stv_stl
2                maxspeed_ohm_net
3    pct_African_American_ohm_net
4              pct_female_ohm_net
5      population_density_ohm_net
6                   slope_ohm_net
7             sep_bikeway_ohm_net
Name: feature, dtype: object

# Charlotte 2 mile buffer

In [379]:
d1=net_data.loc[:,net_data.columns.str.contains('_tm_net', case=False)]
ch_data11=pd.concat([d7, d1.reindex(d7.index)], axis=1)
ch_data11.head()

,AADBT,stv_adb,stv_c_adb,stl_raw,primary_binary,secondary_binary,tertiary_binary,residential_binary,path_binary,cycleway_binary,...,HH_density_tm_net,population_density_tm_net,employment_density_tm_net,Number.of.jobs_tm_net,slope_tm_net,Temp_avg_tm_net,Hum_avg_tm_net,Precipitation_tm_net,sep_bikeway_tm_net,BikeFac_tm_net
102,393.654456,18.547945,5.958904,1236,0,0,0,0,0,1,...,2832.647051,5514.436317,3921.882475,5958.049647,1.549387,63.593151,65.583288,0.148247,5089.053649,0.963836
103,160.742460,16.082192,4.602740,740,0,0,1,0,0,0,...,2401.961584,4781.998045,3558.593176,2386.008587,2.084362,63.593151,65.583288,0.148247,2583.938599,0.489382
104,39.364881,15.465753,2.945205,328,0,0,0,0,0,0,...,1852.537683,4701.628327,2237.932328,835.356120,0.000000,63.593151,65.583288,0.148247,19805.975640,5.135609
105,149.292520,11.630137,4.780822,2049,0,1,0,0,0,0,...,4091.954640,6386.140231,5398.276665,26461.628710,0.000000,63.593151,65.583288,0.148247,6110.769132,1.816037
106,33.307341,2.438356,1.136986,1443,0,1,0,0,0,0,...,1748.502401,2752.939974,1733.910449,9564.959850,2.263852,63.593151,65.583288,0.148247,14287.398580,3.559167


In [380]:
corrMatrix1 = ch_data11.corr()
top_corr_features = corrMatrix1.index[abs(corrMatrix1["AADBT"])>0.20]
top_corr_features

Index(['AADBT', 'stv_adb', 'secondary_binary', 'cycleway_binary',
       'cycleway_track_all_binary', 'Distance.to.Commercial.Area',
       'Distance.to.Commercial.Area.Center', 'Distance.to.Grass',
       'Distance.to.Grass.Center', 'Distance.to.Park',
       'Distance.to.Park.Center', 'Distance.to.Water.Body',
       'Distance.to.forest', 'Distance.to.Forest.Center', 'Distance.to.CBD',
       'min_dist_to_school', 'min_dist_to_university', 'Point.Bridge',
       'stv_nc_adb', 'sep_bikeway_binary', 'sep_onstreet_binary',
       'BikeFac_binary', 'arterial_binary', 'arterial_no_bike_lane_binary',
       'secondary_no_bike_lane_binary', 'log_stv_adb', 'log_stv_nc_adb',
       'log_stv_stl', 'Distance_to_CBD_mi', 'Distance_to_Water_Body_mi',
       'Total.Commuter_tm_net', 'Bike.Commuter_tm_net',
       'Percentage.of.Bike.Commuter_tm_net', 'bike.commuter.density_tm_net',
       'Intersection_Density_tm_net', 'Residential_Road_tm_net',
       'Grass.Area_tm_net', 'Park.Area_tm_net', 'Wat

In [382]:
chdata0=ch_data11[top_corr_features]
chdata00=chdata0.drop(['Distance.to.Grass.Center','cycleway_track_all_binary','secondary_no_bike_lane_binary','arterial_no_bike_lane_binary','arterial_no_bike_lane_binary'],axis=1)

In [383]:
%%time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

chdata00
X_dataf = chdata00.drop("AADBT", axis=1)

while True:
#     # Check the VIF for further filter of the data
# # calculating VIF for each feature
    #X_dataf.replace(0, 1)
    X = add_constant(X_dataf)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
    vif_data=vif_data[1:]
    if vif_data['VIF'].max()<=5:
        break
    vif_data=vif_data.drop([vif_data['VIF'].idxmax()],axis='index')
    X_dataf = chdata00[vif_data['feature'].tolist()]
vif_data.sort_values('VIF')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Wall time: 652 ms


,feature,VIF
6,slope_tm_net,1.272552
1,log_stv_stl,1.384727
3,pct_African_American_tm_net,1.696216
2,maxspeed_tm_net,1.865424
7,sep_bikeway_tm_net,2.194334
4,pct_female_tm_net,2.267989
5,population_density_tm_net,2.352636


In [384]:
vif_data['feature']

1                    log_stv_stl
2                maxspeed_tm_net
3    pct_African_American_tm_net
4              pct_female_tm_net
5      population_density_tm_net
6                   slope_tm_net
7             sep_bikeway_tm_net
Name: feature, dtype: object